# 🔍 AML Investigation Agent

An autonomous AI Agent for investigating financial transaction graphs to identify money laundering patterns.

## Architecture

| Component | Technology |
|-----------|------------|
| Base Model | google/gemma-2-2b-it |
| GPU Support | **CUDA 13.0** (Blackwell/RTX 50 series) |
| Data Processing | **Polars** (fast dataframes) |
| Fine-Tuning | Unsloth + LoRA |
| RL Training | TRL GRPO |
| Agent Framework | Hybrid LangGraph + Model-Driven |
| Observability | MLflow Tracing |
| Evaluation | Gemini LLM-as-Judge (Strategy Quality) |

## Tools

| Tool | Description |
|------|-------------|
| `get_account_summary` | Get account metadata |
| `get_recent_transactions` | Get recent transactions |
| `check_sanctions_list` | Check if entity is sanctioned |
| `submit_suspicious_activity_report` | Submit SAR (terminal action) |

## Key Features

- **LangGraph State Memory**: All short-term memory stored in InvestigationState TypedDict
- **MemorySaver Checkpointing**: LangGraph handles state persistence automatically
- **Strategy-Driven Decisions**: Model uses memory context to adapt investigation strategy
- **Multi-hop Trail Following**: No artificial depth limits
- **Rigorous Evaluation**: LLM-as-judge with outcome-weighted scoring
- **Path Validation**: SAR success requires valid laundering path


## 1. Setup & Dependencies


In [1]:
# Core imports
import os

import json
import random
import time
import re
import uuid
from datetime import datetime
from dataclasses import dataclass, field
from typing import Dict, List, Any, Tuple, Optional, Annotated, Literal
import operator
from dotenv import load_dotenv

import numpy as np
import polars as pl
import pandas as pd  # Keep for compatibility with some libs
import kaggle
import networkx as nx
import torch
import mlflow

# Configuration
MODEL_NAME = "google/gemma-2-2b-it"
GEMINI_MODEL = "gemini-2.0-flash"
MAX_STEPS = 50  # Failsafe limit - agent decides when to stop
EVAL_EPISODES = 10
RANDOM_SEED = 42

# Load environment variables from .env
load_dotenv()

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

print(f"✓ Configuration loaded")
print(f"  Model: {MODEL_NAME}")
print(f"  Max Steps: {MAX_STEPS}")
print(f"  Eval Episodes: {EVAL_EPISODES}")


✓ Configuration loaded
  Model: google/gemma-2-2b-it
  Max Steps: 50
  Eval Episodes: 10


## 2. Load IBM AML Dataset from Kaggle


In [2]:
# Download IBM AML dataset using Kaggle API
DATASET_NAME = "ealtman2019/ibm-transactions-for-anti-money-laundering-aml"

# Use relative path from notebook location (notebooks/agents/) to data/raw/
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, "..", ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
os.makedirs(DATA_DIR, exist_ok=True)

# Expected files from the IBM AML dataset (12 CSVs + 6 pattern files)
EXPECTED_FILES = [
    # Transaction files (6)
    'LI-Small_Trans.csv', 'LI-Medium_Trans.csv', 'LI-Large_Trans.csv',
    'HI-Small_Trans.csv', 'HI-Medium_Trans.csv', 'HI-Large_Trans.csv',
    # Account files (6)
    'LI-Small_accounts.csv', 'LI-Medium_accounts.csv', 'LI-Large_accounts.csv',
    'HI-Small_accounts.csv', 'HI-Medium_accounts.csv', 'HI-Large_accounts.csv',
    # Pattern files (6)
    'LI-Small_Patterns.txt', 'LI-Medium_Patterns.txt', 'LI-Large_Patterns.txt',
    'HI-Small_Patterns.txt', 'HI-Medium_Patterns.txt', 'HI-Large_Patterns.txt',
]

# Check if dataset files already exist (skip download if any expected file is present)
existing_files = os.listdir(DATA_DIR) if os.path.exists(DATA_DIR) else []
dataset_exists = any(f in existing_files for f in EXPECTED_FILES)

if dataset_exists:
    print(f"✓ Dataset already exists in {DATA_DIR}")
    path = DATA_DIR
else:
    print("📥 Downloading IBM AML dataset from Kaggle...")
    try:
        kaggle.api.authenticate()
        kaggle.api.dataset_download_files(DATASET_NAME, path=DATA_DIR, unzip=True, quiet=False)
        print(f"✓ Dataset downloaded to: {DATA_DIR}")
        path = DATA_DIR
    except Exception as e:
        print(f"⚠ Kaggle download failed: {e}")
        print("  Make sure KAGGLE_USERNAME and KAGGLE_KEY are set in .env")
        path = None

# Load transactions AND accounts with Polars for efficiency
if path:
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
    print(f"  Available CSV files: {csv_files}")
    
    # Select dataset size: Small, Medium, or Large (use Small for demo)
    DATASET_SIZE = "Small"  # Options: "Small", "Medium", "Large"
    DATASET_PREFIX = "LI"   # Options: "LI" (Low Illicit), "HI" (High Illicit)
    
    trans_file = os.path.join(path, f"{DATASET_PREFIX}-{DATASET_SIZE}_Trans.csv")
    accounts_file = os.path.join(path, f"{DATASET_PREFIX}-{DATASET_SIZE}_accounts.csv")
    
    if not os.path.exists(trans_file) or not os.path.exists(accounts_file):
        print(f"⚠ Dataset files not found for {DATASET_PREFIX}-{DATASET_SIZE}")
        path = None
    else:
        # Load transactions - handle duplicate "Account" column by providing explicit names
        # Columns: Timestamp, From Bank, Account(from), To Bank, Account(to), Amount Received, 
        #          Receiving Currency, Amount Paid, Payment Currency, Payment Format, Is Laundering
        raw_trans_pl = pl.read_csv(
            trans_file,
            new_columns=['timestamp', 'from_bank', 'from_account', 'to_bank', 'to_account', 
                        'amount_received', 'receiving_currency', 'amount_paid', 
                        'payment_currency', 'payment_format', 'is_laundering'],
            skip_rows=1  # Skip original header
        )
        print(f"✓ Loaded {len(raw_trans_pl)} transactions from {os.path.basename(trans_file)}")
        
        # Load accounts
        # Columns: Bank Name, Bank ID, Account Number, Entity ID, Entity Name
        raw_accounts_pl = pl.read_csv(accounts_file)
        print(f"✓ Loaded {len(raw_accounts_pl)} accounts from {os.path.basename(accounts_file)}")


✓ Dataset already exists in /workspace/data/raw
  Available CSV files: ['HI-Medium_Trans.csv', 'LI-Medium_Trans.csv', 'HI-Large_accounts.csv', 'LI-Large_Trans.csv', 'LI-Small_Trans.csv', 'HI-Small_Trans.csv', 'HI-Medium_accounts.csv', 'LI-Medium_accounts.csv', 'HI-Small_accounts.csv', 'LI-Large_accounts.csv', 'HI-Large_Trans.csv', 'LI-Small_accounts.csv']
✓ Loaded 6924048 transactions from LI-Small_Trans.csv
✓ Loaded 712688 accounts from LI-Small_accounts.csv


In [3]:
# Process IBM AML data with Polars
print("📊 Processing IBM AML data with Polars...")

# === TRANSACTIONS ===
# Create unique account IDs: bank_id + account_number
transactions_pl = raw_trans_pl.with_columns([
    (pl.col('from_bank').cast(pl.Utf8) + '-' + pl.col('from_account').cast(pl.Utf8)).alias('from_account_id'),
    (pl.col('to_bank').cast(pl.Utf8) + '-' + pl.col('to_account').cast(pl.Utf8)).alias('to_account_id'),
    (pl.lit('TXN-') + pl.arange(0, pl.len()).cast(pl.Utf8)).alias('transaction_id'),
    pl.col('is_laundering').cast(pl.Int32),
]).select([
    'transaction_id',
    pl.col('from_account_id').alias('from_account'),
    pl.col('to_account_id').alias('to_account'),
    pl.col('amount_received').alias('amount'),
    pl.col('receiving_currency').alias('currency'),
    'timestamp', 'is_laundering', 'payment_format',
])

# === ACCOUNTS ===
# Identify laundering destinations for sanctioned marking
laundering_dests = set(transactions_pl.filter(pl.col('is_laundering') == 1)['to_account'].unique().to_list())

# Process accounts with sanctioned flag and risk scores
# Entity types in IBM AML data: Corporation, Partnership, Sole Proprietorship
random.seed(RANDOM_SEED)
accounts_pl = raw_accounts_pl.rename({
    'Bank Name': 'bank_name', 'Bank ID': 'bank_id', 
    'Account Number': 'account_number', 'Entity ID': 'entity_id', 'Entity Name': 'entity_name'
}).with_columns([
    (pl.col('bank_id').cast(pl.Utf8) + '-' + pl.col('account_number').cast(pl.Utf8)).alias('account_id'),
    pl.when(pl.col('entity_name').str.contains('Corporation'))
        .then(pl.lit('Corporate'))
        .when(pl.col('entity_name').str.contains('Partnership'))
        .then(pl.lit('Partnership'))
        .when(pl.col('entity_name').str.contains('Sole Proprietorship'))
        .then(pl.lit('Sole Proprietorship'))
        .otherwise(pl.lit('Unknown')).alias('account_type'),
])

# Add sanctioned flag (30% of laundering destinations) and risk scores
account_ids = accounts_pl['account_id'].to_list()
accounts_pl = accounts_pl.with_columns([
    pl.Series('is_sanctioned', [acc in laundering_dests and random.random() < 0.3 for acc in account_ids]),
    pl.Series('risk_score', [round(random.uniform(0.1, 0.9), 2) for _ in range(len(account_ids))]),
]).select(['account_id', 'bank_id', 'bank_name', 'entity_id', 'entity_name', 'account_type', 'risk_score', 'is_sanctioned'])

# Convert to pandas for NetworkX compatibility
transactions_df = transactions_pl.to_pandas()
accounts_df = accounts_pl.to_pandas()

# === DATASET SUMMARY ===
n_accounts = len(accounts_df)
n_transactions = len(transactions_df)
n_laundering = int(transactions_df['is_laundering'].sum())
n_sanctioned = int(accounts_df['is_sanctioned'].sum())

# Count unique accounts actually used in transactions
unique_accounts_in_txns = len(set(transactions_df['from_account'].unique()) | set(transactions_df['to_account'].unique()))

print(f"\n{'='*55}")
print(f"📊 DATASET SUMMARY")
print(f"{'='*55}")
print(f"  Bank Accounts (in file):     {n_accounts:>12,}")
print(f"  Bank Accounts (in txns):     {unique_accounts_in_txns:>12,}")
print(f"  Transactions:                {n_transactions:>12,}")
print(f"  Laundering Transactions:     {n_laundering:>12,}")
print(f"  Laundering Rate:             {n_laundering/n_transactions*100:>11.2f}%")
print(f"{'='*55}")
print(f"  Sanctioned Accounts:         {n_sanctioned:>12,}")
print(f"  Account Types:")
for atype, count in accounts_df['account_type'].value_counts().items():
    print(f"    - {atype:<20} {count:>12,}")
print(f"{'='*55}")


📊 Processing IBM AML data with Polars...

📊 DATASET SUMMARY
  Bank Accounts (in file):          712,688
  Bank Accounts (in txns):          705,907
  Transactions:                   6,924,048
  Laundering Transactions:            3,565
  Laundering Rate:                    0.05%
  Sanctioned Accounts:                  975
  Account Types:
    - Corporate                 268,892
    - Partnership               225,814
    - Sole Proprietorship       217,131
    - Unknown                       851


## 3. Financial Environment with Enhanced Evaluation

The environment validates SAR submissions using path analysis:
- **Primary Targets**: Sanctioned accounts that receive laundering flows
- **Valid SAR**: Entity is sanctioned AND reachable via laundering path from investigation start


In [4]:
@dataclass
class FinancialEnvironment:
    """Financial investigation environment with path-validated SAR evaluation."""
    graph: nx.DiGraph = field(default_factory=nx.DiGraph)
    accounts: Dict[str, dict] = field(default_factory=dict)
    laundering_targets: List[str] = field(default_factory=list)
    all_sanctioned: set = field(default_factory=set)
    laundering_destinations: set = field(default_factory=set)
    current_start_account: str = ""
    
    @classmethod
    def from_dataframes(cls, transactions_df: pd.DataFrame, accounts_df: pd.DataFrame) -> 'FinancialEnvironment':
        env = cls()
        for _, row in accounts_df.iterrows():
            env.accounts[row['account_id']] = row.to_dict()
            env.graph.add_node(row['account_id'], **row.to_dict())
        
        for _, row in transactions_df.iterrows():
            env.graph.add_edge(row['from_account'], row['to_account'],
                transaction_id=row['transaction_id'], amount=row['amount'],
                currency=row.get('currency', 'USD'), timestamp=row['timestamp'],
                is_laundering=row['is_laundering'])
        
        env.all_sanctioned = set(accounts_df[accounts_df['is_sanctioned']]['account_id'])
        laundering_txns = transactions_df[transactions_df['is_laundering'] == 1]
        env.laundering_destinations = set(laundering_txns['to_account'].unique())
        env.laundering_targets = list(env.all_sanctioned & env.laundering_destinations)
        return env
    
    def is_on_laundering_path(self, entity_id: str) -> bool:
        """Check if entity reachable from start via laundering transactions."""
        if not self.current_start_account:
            return False
        try:
            for path in nx.all_simple_paths(self.graph, self.current_start_account, entity_id, cutoff=10):
                if all(self.graph.edges[path[i], path[i+1]].get('is_laundering', 0) == 1 for i in range(len(path)-1)):
                    return True
            return False
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            return False
    
    @mlflow.trace(span_type="TOOL")
    def get_account_summary(self, account_id: str) -> dict:
        if account_id not in self.accounts:
            return {"error": f"Account {account_id} not found"}
        acc = self.accounts[account_id]
        return {
            "account_id": account_id,
            "account_type": acc.get('account_type', 'Unknown'),
            "entity_name": acc.get('entity_name', 'Unknown'),
            "bank_name": acc.get('bank_name', 'Unknown'),
            "risk_score": round(acc.get('risk_score', 0), 2),
            "is_sanctioned": acc.get('is_sanctioned', False),
        }
    
    @mlflow.trace(span_type="TOOL")
    def get_recent_transactions(self, account_id: str, direction: str = "outgoing", limit: int = 5) -> List[dict]:
        if account_id not in self.graph:
            return []
        if direction == "outgoing":
            edges = list(self.graph.out_edges(account_id, data=True))[:limit]
            return [{"to_account": e[1], "amount": e[2].get('amount', 0), "currency": e[2].get('currency', 'USD')} for e in edges]
        else:
            edges = list(self.graph.in_edges(account_id, data=True))[:limit]
            return [{"from_account": e[0], "amount": e[2].get('amount', 0), "currency": e[2].get('currency', 'USD')} for e in edges]
    
    @mlflow.trace(span_type="TOOL")
    def check_sanctions_list(self, entity_id: str) -> dict:
        is_sanctioned = entity_id in self.all_sanctioned
        return {"entity_id": entity_id, "on_sanctions_list": is_sanctioned, 
                "list_type": "OFAC SDN" if is_sanctioned else None}
    
    @mlflow.trace(span_type="TOOL")
    def submit_suspicious_activity_report(self, entity_id: str, reason: str) -> dict:
        is_sanctioned = entity_id in self.all_sanctioned
        is_primary = entity_id in self.laundering_targets
        on_path = self.is_on_laundering_path(entity_id)
        correct = is_primary or (is_sanctioned and on_path)
        
        if is_primary: eval_reason = "PRIMARY_TARGET: sanctioned + receives laundering"
        elif is_sanctioned and on_path: eval_reason = f"VALID: sanctioned + on path from {self.current_start_account}"
        elif is_sanctioned: eval_reason = "INVALID: sanctioned but NOT on laundering path"
        else: eval_reason = "INVALID: not sanctioned"
        
        return {"entity_id": entity_id, "reason": reason, "report_id": f"SAR-{random.randint(10000,99999)}",
                "correct_identification": correct, "is_sanctioned": is_sanctioned,
                "is_primary_target": is_primary, "on_laundering_path": on_path, "evaluation_reason": eval_reason}

env = FinancialEnvironment.from_dataframes(transactions_df, accounts_df)
print(f"✓ Environment: {env.graph.number_of_nodes()} nodes, {env.graph.number_of_edges()} edges")
print(f"  Sanctioned: {len(env.all_sanctioned)}, Primary targets: {len(env.laundering_targets)}")


✓ Environment: 712688 nodes, 1384862 edges
  Sanctioned: 975, Primary targets: 975


## 4. MLflow & Tool Wrappers


In [5]:
# MLflow setup - clean up old trace files to avoid conflicts
import shutil
MLRUNS_DIR = os.path.join(PROJECT_ROOT, "mlruns")
if os.path.isdir(MLRUNS_DIR):
    for exp_name in os.listdir(MLRUNS_DIR):
        traces_path = os.path.join(MLRUNS_DIR, exp_name, "traces")
        if os.path.isdir(traces_path):
            shutil.rmtree(traces_path, ignore_errors=True)
    print("✓ Cleaned old MLflow traces")

mlflow.set_experiment("AML_Investigation_Agent")
print(f"✓ MLflow experiment: AML_Investigation_Agent")

# Tool wrapper functions
def get_account_summary(account_id: str) -> dict:
    return env.get_account_summary(account_id)

def get_recent_transactions(account_id: str, direction: str = "outgoing") -> List[dict]:
    return env.get_recent_transactions(account_id, direction)

def check_sanctions_list(entity_id: str) -> dict:
    return env.check_sanctions_list(entity_id)

def submit_suspicious_activity_report(entity_id: str, reason: str) -> dict:
    return env.submit_suspicious_activity_report(entity_id, reason)

TOOL_MAPPING = {
    "get_account_summary": get_account_summary,
    "get_recent_transactions": get_recent_transactions,
    "check_sanctions_list": check_sanctions_list,
    "submit_suspicious_activity_report": submit_suspicious_activity_report,
}

# Robust tool call parsing - handles multiple formats
def extract_thinking(text: str) -> str:
    """Extract thinking content from model output (for logging/analysis)."""
    match = re.search(r'<thinking>(.*?)</thinking>', text, re.DOTALL | re.IGNORECASE)
    return match.group(1).strip() if match else ""

def extract_tool_calls(text: str) -> List[dict]:
    """Extract tool calls from model output - supports multiple formats.
    
    Handles FunctionGemma-style <thinking> tags by stripping them before parsing.
    """
    calls = []
    
    # Strip thinking tags before parsing ACTION/PARAMS
    clean_text = re.sub(r'<thinking>.*?</thinking>', '', text, flags=re.DOTALL | re.IGNORECASE)
    
    # Format 1: ACTION: tool_name\nPARAMS: {...} (preferred)
    action_match = re.search(r'ACTION:\s*(\w+)', clean_text, re.IGNORECASE)
    if action_match:
        tool_name = action_match.group(1)
        args = {}
        
        # Try to extract JSON params
        params_match = re.search(r'PARAMS:\s*(\{[^}]+\})', clean_text, re.IGNORECASE | re.DOTALL)
        if params_match:
            try:
                args = json.loads(params_match.group(1))
            except:
                # Parse key:value pairs
                for kv in re.finditer(r'"?(\w+)"?\s*:\s*"?([^",}\n]+)"?', params_match.group(1)):
                    args[kv.group(1)] = kv.group(2).strip().strip('"\'')
        else:
            # Try inline params: ACTION: tool_name(param=value)
            inline_match = re.search(r'ACTION:\s*\w+\s*[(\{]([^)\}]+)[)\}]', clean_text, re.IGNORECASE)
            if inline_match:
                for kv in re.finditer(r'(\w+)\s*[=:]\s*"?([^",)\}\n]+)"?', inline_match.group(1)):
                    args[kv.group(1)] = kv.group(2).strip().strip('"\'')
        
        calls.append({"name": tool_name, "arguments": args})
        return calls
    
    # Format 2: FunctionGemma format <start_function_call>call:func{args}<end_function_call>
    for match in re.finditer(r"<start_function_call>call:(\w+)\{(.*?)\}<end_function_call>", text, re.DOTALL):
        name, args_str = match.groups()
        args = {}
        for arg_match in re.finditer(r"(\w+):([^,}]+)", args_str):
            val = arg_match.group(2).strip().strip('"\'')
            try: val = int(val)
            except:
                try: val = float(val)
                except: pass
            args[arg_match.group(1)] = val
        calls.append({"name": name, "arguments": args})
    
    # Format 3: Simple function call syntax: tool_name(arg1="val1", arg2="val2")
    if not calls:
        for match in re.finditer(r'\b(get_account_summary|get_recent_transactions|check_sanctions_list|submit_suspicious_activity_report)\s*\(([^)]*)\)', text):
            name = match.group(1)
            args = {}
            for kv in re.finditer(r'(\w+)\s*=\s*"?([^",)]+)"?', match.group(2)):
                args[kv.group(1)] = kv.group(2).strip().strip('"\'')
            calls.append({"name": name, "arguments": args})
    
    return calls

def execute_tool_call(call: dict) -> Any:
    """Execute a tool call with argument name normalization."""
    fn = TOOL_MAPPING.get(call.get('name'))
    if fn:
        args = call.get('arguments', {}).copy()
        tool_name = call.get('name')
        
        # Normalize argument names: entity_id -> account_id for account-based tools
        if tool_name in ('get_account_summary', 'get_recent_transactions'):
            if 'entity_id' in args and 'account_id' not in args:
                args['account_id'] = args.pop('entity_id')
        
        # Ensure required arguments exist
        if tool_name == 'get_account_summary' and 'account_id' not in args:
            return {"error": "Missing account_id argument"}
        if tool_name == 'get_recent_transactions' and 'account_id' not in args:
            return {"error": "Missing account_id argument"}
        if tool_name == 'check_sanctions_list' and 'entity_id' not in args:
            # Try account_id as fallback
            if 'account_id' in args:
                args['entity_id'] = args.pop('account_id')
            else:
                return {"error": "Missing entity_id argument"}
        if tool_name == 'submit_suspicious_activity_report':
            if 'entity_id' not in args:
                if 'account_id' in args:
                    args['entity_id'] = args.pop('account_id')
                else:
                    return {"error": "Missing entity_id argument"}
            if 'reason' not in args:
                args['reason'] = "Suspicious activity detected"
        
        try:
            return fn(**args)
        except TypeError as e:
            return {"error": f"Tool call error: {e}"}
    return {"error": f"Unknown tool: {call.get('name')}"}

print("✓ Tools configured:", list(TOOL_MAPPING.keys()))


✓ MLflow experiment: AML_Investigation_Agent
✓ Tools configured: ['get_account_summary', 'get_recent_transactions', 'check_sanctions_list', 'submit_suspicious_activity_report']


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


## 5. Load Model with Unsloth

Unsloth provides optimized model loading with 4-bit quantization and Triton-accelerated kernels. With CUDA 13.0, Blackwell GPUs (RTX 50 series) are fully supported.


In [6]:
# Load model with Unsloth
from unsloth import FastLanguageModel

print("📥 Loading model with Unsloth...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=16384,  # Extended for longer prompts with thinking
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print(f"✓ Model loaded: {MODEL_NAME}")
print(f"  Parameters: {model.num_parameters():,}")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
📥 Loading model with Unsloth...
==((====))==  Unsloth 2025.12.9: Fast Gemma2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0a0+b558c986e8.nv25.11. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[unsloth_zoo.log|WARNING]Unsloth: unsloth/gemma-2-2b-it-bnb-4bit can only handle sequence lengths of at most 8192.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 16384!


✓ Model loaded: google/gemma-2-2b-it
  Parameters: 2,614,341,888


## 6. LangGraph Agent with Integrated State Memory

The agent uses **LangGraph** with **MemorySaver** for state management:

### InvestigationState (TypedDict)
All short-term memory is stored directly in the LangGraph state:
- **accounts_analyzed**: Account summaries examined
- **entities_checked**: Sanctions check results
- **risk_indicators**: Risk signals discovered
- **investigation_path**: Path of accounts followed
- **total_amount_traced**: Running transaction total
- **current_strategy**: Dynamic strategy (explore → follow_risk → submit_report)

### Key Components
- **MemorySaver**: Checkpoints state at each step for recovery
- **model_decision_node**: Where LLM makes tool-calling decisions
- **InvestigationEpisode**: Stores trajectory for evaluation


In [7]:
# LangGraph for structure + checkpointing, MODEL for decisions
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

# ============================================================================
# InvestigationState: All memory is stored directly in LangGraph state
# MemorySaver handles persistence/checkpointing automatically
# ============================================================================

class InvestigationState(TypedDict):
    """
    Complete state for LangGraph agent - ALL memory is stored here.
    LangGraph's MemorySaver persists this state for checkpointing.
    """
    # Investigation target
    start_account: str
    
    # SHORT-TERM MEMORY (managed by LangGraph)
    accounts_analyzed: Dict[str, dict]    # account_id -> summary
    entities_checked: Dict[str, bool]     # entity_id -> is_sanctioned
    risk_indicators: List[str]            # Risk signals discovered
    investigation_path: List[str]         # Path of accounts followed
    total_amount_traced: float            # Running total of amounts
    current_strategy: str                 # explore/follow_risk/follow_amount/submit_report
    
    # Conversation state
    history: List[dict]                   # Assistant/tool message history
    
    # Execution tracking
    steps: List[dict]                     # Investigation step records
    step_count: int
    terminated: bool
    success: bool
    final_result: dict

def get_memory_context(state: InvestigationState) -> str:
    """
    Build memory context string from LangGraph state for model prompt.
    
    IMPORTANT: Format must EXACTLY match SFT training samples!
    Always include Progress, Path, and Strategy lines.
    """
    ctx = [f"Progress: {len(state['accounts_analyzed'])} accounts | ${state['total_amount_traced']:,.0f} traced"]
    
    # ALWAYS include Path line (even when empty) - matches SFT training format
    if state['investigation_path']:
        ctx.append(f"Path: {' → '.join(state['investigation_path'][-4:])}")
    else:
        ctx.append("Path: (starting)")
    
    # Only include Risks if present
    if state['risk_indicators']:
        # Simplify risk indicators to match SFT format
        simplified_risks = []
        for r in state['risk_indicators'][-3:]:
            # Remove account IDs from risk indicators for cleaner format
            if ':' in r:
                simplified_risks.append(r.split(':')[0])
            else:
                simplified_risks.append(r)
        ctx.append(f"Risks: {'; '.join(simplified_risks)}")
    
    ctx.append(f"Strategy: {state['current_strategy'].upper()}")
    return "\n".join(ctx)

def update_strategy(state: InvestigationState) -> str:
    """Determine investigation strategy based on current state."""
    if any("SANCTIONED" in r for r in state['risk_indicators']):
        return "submit_report"
    elif any("HIGH_RISK" in r or "LARGE" in r for r in state['risk_indicators']):
        return "follow_risk"
    elif state['total_amount_traced'] > 100000:
        return "follow_amount"
    return "explore"

def update_state_from_tool_result(state: InvestigationState, tool_name: str, args: dict, result: Any) -> InvestigationState:
    """Update LangGraph state based on tool execution result."""
    # Create new state (LangGraph states should be treated as immutable)
    new_state = dict(state)
    new_state['accounts_analyzed'] = dict(state['accounts_analyzed'])
    new_state['entities_checked'] = dict(state['entities_checked'])
    new_state['risk_indicators'] = list(state['risk_indicators'])
    new_state['investigation_path'] = list(state['investigation_path'])
    
    if tool_name == "get_account_summary":
        acc_id = args.get("account_id", "")
        new_state['accounts_analyzed'][acc_id] = result if isinstance(result, dict) else {}
        if acc_id not in new_state['investigation_path']:
            new_state['investigation_path'].append(acc_id)
        if isinstance(result, dict):
            if result.get("risk_score", 0) > 0.7:
                new_state['risk_indicators'].append(f"HIGH_RISK_SCORE:{acc_id}:{result.get('risk_score')}")
            if result.get("is_sanctioned"):
                new_state['risk_indicators'].append(f"SANCTIONED_ACCOUNT:{acc_id}")
    
    elif tool_name == "get_recent_transactions":
        new_state['risk_indicators'].append("TXN_RETRIEVED")  # Mark that we got transactions
        if isinstance(result, list):
            for txn in result:
                new_state['total_amount_traced'] += txn.get("amount", 0)
                to_account = txn.get("to_account")
                if to_account and to_account not in new_state['accounts_analyzed']:
                    # Track new account found in transactions
                    new_state['accounts_analyzed'][to_account] = {"from_transaction": True}
                    if to_account not in new_state['investigation_path']:
                        new_state['investigation_path'].append(to_account)
                if txn.get("amount", 0) > 100000:
                    new_state['risk_indicators'].append(f"LARGE_TXN:{to_account}:{txn.get('amount')}")
    
    elif tool_name == "check_sanctions_list":
        entity_id = args.get("entity_id", "")
        is_sanctioned = result.get("on_sanctions_list", False) if isinstance(result, dict) else False
        new_state['entities_checked'][entity_id] = is_sanctioned
        if is_sanctioned:
            new_state['risk_indicators'].append(f"SANCTIONED:{entity_id}")
    
    # Update strategy based on new state
    new_state['current_strategy'] = update_strategy(new_state)
    
    return new_state

print("✓ InvestigationState defined with integrated memory")
print("  - All memory stored directly in LangGraph state")
print("  - MemorySaver handles checkpointing automatically")

@dataclass
class InvestigationEpisode:
    """Records an investigation episode for evaluation."""
    start_account: str
    steps: List[dict] = field(default_factory=list)
    terminated: bool = False
    success: bool = False
    final_result: dict = field(default_factory=dict)
    
    def add_step(self, action: str, tool_call: dict, result: Any):
        self.steps.append({"step": len(self.steps) + 1, "action": action, "tool_call": tool_call, "result": result})
    
    @classmethod
    def from_state(cls, state: dict) -> 'InvestigationEpisode':
        episode = cls(start_account=state.get('start_account', ''))
        episode.steps = state.get('steps', [])
        episode.terminated = state.get('terminated', False)
        episode.success = state.get('success', False)
        episode.final_result = state.get('final_result', {})
        return episode
    
    def print_trace(self):
        print(f"\\n{'─'*60}")
        print(f"📋 INVESTIGATION TRACE: {self.start_account}")
        print(f"{'─'*60}")
        for step in self.steps:
            tool_name = step.get('tool_call', {}).get('name', 'unknown')
            args = step.get('tool_call', {}).get('arguments', {})
            result = step.get('result', {})
            
            if tool_name == "get_account_summary":
                print(f"  [{step['step']}] 👤 {tool_name}({args.get('account_id', '?')})")
                if isinstance(result, dict) and 'error' not in result:
                    sanctioned_flag = "⚠️ SANCTIONED" if result.get('is_sanctioned') else ""
                    print(f"      → {result.get('account_type')} | {result.get('entity_name', 'Unknown')[:30]} | Risk: {result.get('risk_score')} {sanctioned_flag}")
            elif tool_name == "get_recent_transactions":
                print(f"  [{step['step']}] 💸 {tool_name}({args.get('account_id', '?')})")
                if isinstance(result, list):
                    for txn in result[:3]:
                        print(f"      → {txn.get('to_account', '?')}: ${txn.get('amount', 0):,.2f}")
            elif tool_name == "check_sanctions_list":
                sanctioned = result.get('on_sanctions_list', False) if isinstance(result, dict) else False
                print(f"  [{step['step']}] 🔍 {tool_name}({args.get('entity_id', '?')}) → {'⚠️ SANCTIONED' if sanctioned else '✓ Clear'}")
            elif tool_name == "submit_suspicious_activity_report":
                if isinstance(result, dict):
                    print(f"  [{step['step']}] 📝 {tool_name}({args.get('entity_id', '?')})")
                    print(f"      → {'✅ CORRECT' if result.get('correct_identification') else '❌ INCORRECT'}")
                    print(f"      → {result.get('evaluation_reason', '')}")
        
        print(f"{'─'*60}")
        print(f"  Result: {'✅ SUCCESS' if self.success else '❌ FAILED'} | Steps: {len(self.steps)}")
        print(f"{'─'*60}")

print("✓ InvestigationEpisode class defined")


✓ InvestigationState defined with integrated memory
  - All memory stored directly in LangGraph state
  - MemorySaver handles checkpointing automatically
✓ InvestigationEpisode class defined


In [8]:
# ============================================================================
# HYBRID LANGGRAPH AGENT: Structure + Checkpointing, MODEL-DRIVEN Decisions
# Memory is stored DIRECTLY in LangGraph state (no separate class needed)
# ============================================================================

# Global references for model/tokenizer (set before running agent)
_model = None
_tokenizer = None

def set_model_for_agent(model, tokenizer):
    """Set the model and tokenizer for the LangGraph agent."""
    global _model, _tokenizer
    _model, _tokenizer = model, tokenizer

# Tool description with thinking capability (FunctionGemma style)
# The model reasons before acting, improving decision quality
TOOL_DESC = """You are an AML investigator. Use these tools to investigate money laundering:

TOOLS:
- get_account_summary(account_id): Get account info and risk score
- get_recent_transactions(account_id, direction): Get transactions (direction: "outgoing" or "incoming")
- check_sanctions_list(entity_id): Check if entity is on sanctions list
- submit_suspicious_activity_report(entity_id, reason): Submit SAR (ONLY if entity is confirmed sanctioned!)

RESPONSE FORMAT:
<thinking>Brief reasoning about your next action</thinking>
ACTION: tool_name
PARAMS: {"key": "value"}

RULES:
- Always use <thinking> before ACTION to explain your reasoning
- NEVER submit SAR unless check_sanctions_list confirmed the entity is sanctioned
- Follow money trails through transactions to find suspicious entities"""

def build_prompt_from_state(state: InvestigationState) -> str:
    """
    Build model prompt with memory context from LangGraph state.
    
    IMPORTANT: Do NOT prefill model response - let the model generate the full
    response including ACTION and PARAMS. This matches SFT training format exactly.
    """
    memory_ctx = get_memory_context(state)
    
    # Build base prompt
    prompt = f"<start_of_turn>user\nInvestigate account {state['start_account']} for money laundering.\n\n{TOOL_DESC}\n\nCURRENT STATUS:\n{memory_ctx}<end_of_turn>\n"
    
    # Add conversation history (limit to last 4 exchanges to prevent prompt explosion)
    history = state['history'][-8:]  # 4 exchanges = 8 messages (assistant + tool each)
    for msg in history:
        if msg['role'] == 'assistant':
            # Truncate long model responses
            content = msg['content'][:300]
            prompt += f"<start_of_turn>model\n{content}<end_of_turn>\n"
        elif msg['role'] == 'tool':
            # Truncate long tool results
            content = msg['content'][:400]
            prompt += f"<start_of_turn>user\nTOOL RESULT: {content}<end_of_turn>\n"
    
    # Start model turn - let model generate full response (no prefilling!)
    # This matches SFT training format exactly
    prompt += "<start_of_turn>model\n"
    
    return prompt

# Global verbose flag for detailed logging
_verbose_logging = False

@mlflow.trace(span_type="AGENT_NODE")
def model_decision_node(state: InvestigationState) -> InvestigationState:
    """
    MODEL-DRIVEN NODE: The model decides which tool to call.
    All memory is stored directly in LangGraph state - no separate class needed.
    """
    global _model, _tokenizer, _verbose_logging
    
    start_account = state["start_account"]
    step_num = state["step_count"] + 1
    
    # Build prompt with memory context from state
    prompt = build_prompt_from_state(state)
    
    # Truncate prompt to prevent exceeding max_seq_length
    MAX_PROMPT_TOKENS = 15000  # Leave room for generation (256 tokens)
    inputs = _tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_PROMPT_TOKENS).to(_model.device)
    
    # MODEL GENERATES (non-deterministic with temperature)
    with torch.no_grad():
        outputs = _model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=_tokenizer.eos_token_id
        )
    
    response = _tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=False)
    raw_response = response  # Keep for logging
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0]
    
    # Extract thinking and tool call from model response
    thinking = extract_thinking(response)
    tool_calls = extract_tool_calls(response)
    used_fallback = False
    
    # Detailed logging if enabled
    if _verbose_logging:
        print(f"\n{'='*60}")
        print(f"🔬 STEP {step_num} DEBUG")
        print(f"{'='*60}")
        if thinking:
            print(f"💭 THINKING: {thinking[:200]}{'...' if len(thinking) > 200 else ''}")
        print(f"📥 RAW MODEL OUTPUT ({len(raw_response)} chars):")
        print(f"---\n{response[:500]}{'...' if len(response) > 500 else ''}\n---")
        print(f"📤 EXTRACTED TOOL CALLS: {tool_calls}")
    
    # If no valid tool call, use strategy-based fallback to continue investigation
    if not tool_calls:
        used_fallback = True
        if _verbose_logging:
            print(f"⚠️ NO VALID TOOL CALL - using fallback strategy")
        
        # Strategy-based next action based on current state
        accounts_analyzed = state.get("accounts_analyzed", {})
        entities_checked = state.get("entities_checked", {})
        risk_indicators = state.get("risk_indicators", [])
        path = state.get("investigation_path", [])
        
        if state["step_count"] == 0:
            # First step: always get account summary
            tool_calls = [{"name": "get_account_summary", "arguments": {"account_id": start_account}}]
        
        elif any("SANCTIONED" in r for r in risk_indicators):
            # Found sanctioned entity - submit SAR (ONLY if we have confirmed sanctioned entity)
            sanctioned = [r.split(":")[1] for r in risk_indicators if "SANCTIONED" in r and ":" in r]
            if sanctioned:
                # Double-check: only submit if entity was marked as sanctioned
                sanctioned_entity = sanctioned[-1]
                tool_calls = [{"name": "submit_suspicious_activity_report", "arguments": {"entity_id": sanctioned_entity, "reason": "Sanctioned entity confirmed on OFAC list"}}]
        
        # Check if we need to get transactions
        if not tool_calls and not any("TXN_RETRIEVED" in r for r in risk_indicators):
            # Haven't retrieved transactions yet
            tool_calls = [{"name": "get_recent_transactions", "arguments": {"account_id": start_account, "direction": "outgoing"}}]
        
        # Check sanctions on accounts we found
        if not tool_calls:
            unchecked = [acc for acc in accounts_analyzed if acc not in entities_checked and acc != start_account]
            if unchecked:
                tool_calls = [{"name": "check_sanctions_list", "arguments": {"entity_id": unchecked[0]}}]
        
        # Follow transaction trail
        if not tool_calls and len(path) > 1:
            # Get transactions for accounts we haven't fully explored
            for acc in path[1:]:  # Skip start account
                if acc not in entities_checked:
                    tool_calls = [{"name": "check_sanctions_list", "arguments": {"entity_id": acc}}]
                    break
        
        # Final fallback: terminate if no action possible or too many steps
        if not tool_calls or state["step_count"] >= 15:
            return {**state, "terminated": True, "success": False}
    
    # Execute the first tool call
    call = tool_calls[0]
    
    # CRITICAL: Validate SAR submission - only allow if entity was confirmed sanctioned
    if call.get("name") == "submit_suspicious_activity_report":
        entity_to_report = call.get("arguments", {}).get("entity_id")
        entities_checked = state.get("entities_checked", {})
        risk_indicators = state.get("risk_indicators", [])
        
        # Check if this entity was confirmed as sanctioned
        entity_is_sanctioned = entities_checked.get(entity_to_report, False)
        has_sanctioned_indicator = any(f"SANCTIONED:{entity_to_report}" in r or f"SANCTIONED_ACCOUNT:{entity_to_report}" in r for r in risk_indicators)
        
        if not entity_is_sanctioned and not has_sanctioned_indicator:
            # Entity not confirmed sanctioned - redirect to check sanctions first
            if _verbose_logging:
                print(f"⚠️ SAR BLOCKED: {entity_to_report} not confirmed sanctioned - checking first")
            call = {"name": "check_sanctions_list", "arguments": {"entity_id": entity_to_report}}
    
    result = execute_tool_call(call)
    
    # Update state with tool result (memory is in state, not separate class)
    updated_state = update_state_from_tool_result(state, call.get("name", ""), call.get("arguments", {}), result)
    
    # Logging after execution
    if _verbose_logging:
        if used_fallback:
            print(f"🔄 FALLBACK ACTION: {call.get('name')}({call.get('arguments', {})})")
        else:
            print(f"✅ MODEL ACTION: {call.get('name')}({call.get('arguments', {})})")
        result_str = str(result)[:200] + "..." if len(str(result)) > 200 else str(result)
        print(f"📤 RESULT: {result_str}")
    
    # Build step record with memory context and thinking (for analysis)
    step = {
        "step": state["step_count"] + 1,
        "action": call.get("name"),
        "tool_call": call,
        "result": result,
        "model_response": response,
        "thinking": thinking,  # FunctionGemma-style reasoning (for analysis)
        "memory_state": get_memory_context(updated_state),
        "used_fallback": used_fallback,  # Track if model made decision or fallback was used
    }
    
    # Check if terminal action
    terminated = False
    success = False
    final_result = {}
    
    if call.get("name") == "submit_suspicious_activity_report":
        terminated = True
        success = result.get("correct_identification", False) if isinstance(result, dict) else False
        final_result = result
    
    # Update history
    new_history = list(state["history"]) + [
        {"role": "assistant", "content": response},
        {"role": "tool", "content": json.dumps(result)}
    ]
    
    # Return updated state (all memory is in the state dict)
    return {
        **updated_state,
        "history": new_history,
        "steps": list(state["steps"]) + [step],
        "step_count": state["step_count"] + 1,
        "terminated": terminated,
        "success": success,
        "final_result": final_result,
    }

def should_continue(state: InvestigationState) -> str:
    """Routing function: continue or end based on state."""
    if state["terminated"]:
        return END
    if state["step_count"] >= MAX_STEPS:
        return END
    return "model_decision"

# Build LangGraph with MemorySaver
def build_hybrid_agent():
    """Build the hybrid LangGraph agent with model-driven decisions."""
    workflow = StateGraph(InvestigationState)
    
    # Single node where MODEL makes decisions
    workflow.add_node("model_decision", model_decision_node)
    
    # Entry point
    workflow.add_edge(START, "model_decision")
    
    # Conditional continuation
    workflow.add_conditional_edges(
        "model_decision",
        should_continue,
        {"model_decision": "model_decision", END: END}
    )
    
    # Compile with MemorySaver for checkpointing
    checkpointer = MemorySaver()
    return workflow.compile(checkpointer=checkpointer)

# Build the agent
investigation_agent = build_hybrid_agent()

print("✓ Hybrid LangGraph Agent built")
print("  - Structure: LangGraph StateGraph with MemorySaver")
print("  - Decisions: MODEL-DRIVEN (non-deterministic)")
print("  - Checkpointing: Enabled via MemorySaver")


✓ Hybrid LangGraph Agent built
  - Structure: LangGraph StateGraph with MemorySaver
  - Decisions: MODEL-DRIVEN (non-deterministic)
  - Checkpointing: Enabled via MemorySaver


## 7. Model-Driven Investigation with LangGraph State Memory

The model sees rich context from `InvestigationState` (managed by LangGraph's `MemorySaver`) to make strategic decisions. All memory fields are stored directly in the LangGraph state:

- **accounts_analyzed**: Summary of each account examined
- **entities_checked**: Sanctions status of checked entities
- **risk_indicators**: Risk signals discovered during investigation
- **investigation_path**: Path of accounts followed
- **total_amount_traced**: Running total of transaction amounts
- **current_strategy**: Dynamic strategy (explore → follow_risk → submit_report)


In [9]:
# ============================================================================
# RUN MODEL-DRIVEN EPISODE (uses LangGraph agent with integrated memory)
# ============================================================================
# NOTE: Memory is stored DIRECTLY in InvestigationState (defined in cell 13)
# - accounts_analyzed, entities_checked, risk_indicators, investigation_path, etc.
# - LangGraph's MemorySaver handles checkpointing of entire state
# - No separate InvestigationMemory class needed
# ============================================================================

def create_initial_state(start_account: str) -> InvestigationState:
    """Create fresh InvestigationState with all memory fields initialized."""
    return {
        "start_account": start_account,
        # Short-term memory (managed by LangGraph)
        "accounts_analyzed": {},
        "entities_checked": {},
        "risk_indicators": [],
        "investigation_path": [],
        "total_amount_traced": 0.0,
        "current_strategy": "explore",
        # Conversation state
        "history": [],
        # Execution tracking
        "steps": [],
        "step_count": 0,
        "terminated": False,
        "success": False,
        "final_result": {},
    }

@mlflow.trace(name="langgraph_investigation", span_type="CHAIN")
def run_model_driven_episode(model, tokenizer, start_account: str,
                              max_steps: int = MAX_STEPS, verbose: bool = True) -> InvestigationEpisode:
    """
    Run investigation using HYBRID LANGGRAPH agent.
    - LangGraph provides structure, state tracking, and checkpointing via MemorySaver
    - All memory is stored directly in InvestigationState (no separate class)
    - MODEL makes non-deterministic decisions based on memory context
    """
    # Set model for the LangGraph agent node
    set_model_for_agent(model, tokenizer)
    env.current_start_account = start_account
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"🔍 LANGGRAPH INVESTIGATION: {start_account}")
        print(f"   Memory managed by: LangGraph MemorySaver")
        print(f"{'='*70}")
    
    # Create initial state with all memory fields (directly in state)
    initial_state = create_initial_state(start_account)
    
    # Unique thread ID for checkpointing
    thread_id = f"investigation_{start_account}_{uuid.uuid4().hex[:8]}"
    config = {"configurable": {"thread_id": thread_id}, "recursion_limit": MAX_STEPS + 10}
    
    try:
        # Run the LangGraph agent
        final_state = investigation_agent.invoke(initial_state, config=config)
        
        # Print step-by-step trace if verbose
        if verbose:
            for step in final_state.get("steps", []):
                step_num = step.get("step", "?")
                tool_name = step.get("tool_call", {}).get("name", "unknown")
                args = step.get("tool_call", {}).get("arguments", {})
                result = step.get("result", {})
                model_response = step.get("model_response", "")[:100]
                memory_state = step.get("memory_state", "")[:80]
                
                print(f"\n┌─ Step {step_num} {'─'*55}")
                print(f"│ 📝 MODEL: {model_response}...")
                print(f"│ 🧠 MEMORY (LangGraph state): {memory_state}")
                
                args_str = ", ".join(f"{k}={v}" for k, v in args.items())
                result_preview = json.dumps(result)[:80] if isinstance(result, (dict, list)) else str(result)[:80]
                print(f"│ 🔧 TOOL: {tool_name}({args_str})")
                print(f"│ 📤 RESULT: {result_preview}...")
            
            # Final status
            success = final_state.get("success", False)
            step_count = final_state.get("step_count", 0)
            final_result = final_state.get("final_result", {})
            
            # Show final memory state
            print(f"\n┌─ Final Memory State {'─'*45}")
            print(f"│ Accounts analyzed: {len(final_state.get('accounts_analyzed', {}))}")
            print(f"│ Entities checked: {len(final_state.get('entities_checked', {}))}")
            print(f"│ Amount traced: ${final_state.get('total_amount_traced', 0):,.0f}")
            print(f"│ Risk indicators: {len(final_state.get('risk_indicators', []))}")
            print(f"│ Strategy: {final_state.get('current_strategy', 'unknown')}")
            
            if final_state.get("terminated") and final_result:
                status = "✅ SUCCESS" if success else "❌ FAILED"
                print(f"\n└─ {status}: {final_result.get('evaluation_reason', 'N/A')}")
                print(f"   Thread: {thread_id} | Steps: {step_count}")
            else:
                print(f"\n└─ ⏱️ MAX STEPS ({step_count}) or NO SAR")
        
        # Convert LangGraph state to InvestigationEpisode
        episode = InvestigationEpisode(start_account=start_account)
        episode.steps = final_state.get("steps", [])
        episode.terminated = final_state.get("terminated", False)
        episode.success = final_state.get("success", False)
        episode.final_result = final_state.get("final_result", {})
        
        return episode
        
    except Exception as e:
        if verbose:
            print(f"└─ ❌ Agent error: {e}")
            import traceback
            traceback.print_exc()
        return InvestigationEpisode(start_account=start_account)

print("✓ Hybrid LangGraph run_model_driven_episode defined")
print("  - Uses investigation_agent (LangGraph with MemorySaver)")
print("  - Model decisions via model_decision_node")

✓ Hybrid LangGraph run_model_driven_episode defined
  - Uses investigation_agent (LangGraph with MemorySaver)
  - Model decisions via model_decision_node


## 8. Gemini LLM-as-Judge (Strategy Evaluation)

Rigorous evaluation where FAILED investigations are capped at score 2 regardless of apparent strategy quality.


In [10]:
# Use new google.genai library (replaces deprecated google.generativeai)
from google import genai

# Initialize Gemini client with new API
gemini_client = None
try:
    api_key = os.environ.get("GEMINI_API_KEY", os.environ.get("GOOGLE_API_KEY", ""))
    if api_key:
        gemini_client = genai.Client(api_key=api_key)
        print(f"✓ Gemini client initialized: {GEMINI_MODEL}")
    else:
        print("⚠ No GEMINI_API_KEY or GOOGLE_API_KEY found in environment")
except Exception as e:
    print(f"⚠ Gemini not available: {e}")

def evaluate_investigation_with_gemini(episode: InvestigationEpisode, ground_truth_target: str) -> dict:
    """Evaluate TRAJECTORY QUALITY - how well did the agent use memory and strategy to investigate."""
    tool_names = [s['tool_call'].get('name') for s in episode.steps]
    unique_tools = set(tool_names)
    num_steps = len(episode.steps)
    
    has_sar = 'submit_suspicious_activity_report' in unique_tools
    proper_flow = 'get_account_summary' in unique_tools and 'get_recent_transactions' in unique_tools
    is_multi_hop = tool_names.count('get_recent_transactions') > 1
    checked_sanctions = 'check_sanctions_list' in unique_tools
    
    sar_result = None
    for step in episode.steps:
        if step['tool_call'].get('name') == 'submit_suspicious_activity_report':
            sar_result = step.get('result', {})
            break
    
    # Extract memory/strategy usage from steps
    memory_states = [s.get('memory_state', '') for s in episode.steps if s.get('memory_state')]
    strategy_changes = len(set(memory_states))  # How many times strategy evolved
    
    if not gemini_client:
        # Rigorous fallback scoring based on OUTCOME + TRAJECTORY QUALITY
        if episode.success:
            score = 5
            strategy_quality = 5 if strategy_changes > 1 else (4 if proper_flow else 3)
            context_usage = 5 if memory_states else 3
            trail_following = 5 if is_multi_hop else 4
            efficiency = 5 if num_steps <= 5 else (4 if num_steps <= 10 else 3)
            explanation = f"SUCCESS: Found target via {'multi-hop' if is_multi_hop else 'direct'} path in {num_steps} steps"
        elif episode.terminated and has_sar:
            is_sanctioned = sar_result.get('is_sanctioned', False) if sar_result else False
            if is_sanctioned:
                score, strategy_quality = 2, 2
                context_usage = 2 if memory_states else 1
                trail_following, efficiency = 2, 2
                explanation = "PARTIAL: Found sanctioned entity but not on valid laundering path"
            else:
                score, strategy_quality, context_usage = 1, 1, 1
                trail_following, efficiency = 1, 1
                explanation = "FAILED: Reported non-sanctioned entity"
        else:
            score = 1
            strategy_quality = 2 if proper_flow else 1
            context_usage = 2 if memory_states else 1
            trail_following = 2 if is_multi_hop else 1
            efficiency = 1
            explanation = f"FAILED: No SAR submitted after {num_steps} steps"
        
        return {"score": score, "strategy_quality": strategy_quality, "context_usage": context_usage,
                "trail_following": trail_following, "efficiency": efficiency, "explanation": explanation}
    
    # Build detailed trajectory for Gemini evaluation
    trajectory_details = []
    for i, step in enumerate(episode.steps):
        tool = step['tool_call'].get('name', '?')
        args = step['tool_call'].get('arguments', {})
        result = step.get('result', {})
        memory = step.get('memory_state', 'No memory')
        
        if tool == "submit_suspicious_activity_report":
            correct = result.get('correct_identification', False) if isinstance(result, dict) else False
            trajectory_details.append(f"Step {i+1}: SAR({args.get('entity_id')}) → {'✓ CORRECT' if correct else '✗ WRONG'}")
        elif tool == "check_sanctions_list":
            sanctioned = result.get('on_sanctions_list', False) if isinstance(result, dict) else False
            trajectory_details.append(f"Step {i+1}: check_sanctions({args.get('entity_id')}) → {'SANCTIONED' if sanctioned else 'Clear'}")
        else:
            trajectory_details.append(f"Step {i+1}: {tool}({list(args.values())[0] if args else ''})")
        
        if memory and i < 3:  # Show memory for first few steps
            trajectory_details.append(f"         Memory: {memory[:60]}")
    
    sar_details = ""
    if sar_result:
        sar_details = f"""
SAR Evaluation:
- Entity: {sar_result.get('entity_id')}
- Sanctioned: {sar_result.get('is_sanctioned')}
- On Laundering Path: {sar_result.get('on_laundering_path')}
- Reason: {sar_result.get('evaluation_reason')}"""
    
    prompt = f'''You are evaluating an AML investigation agent's TRAJECTORY QUALITY.

OUTCOME: {"✓ SUCCESS" if episode.success else "✗ FAILED"} after {num_steps} steps
{sar_details}

TRAJECTORY:
{chr(10).join(trajectory_details)}

Evaluate the STRATEGY used (not just outcome):
1. strategy_quality: Did agent follow logical investigation flow? (account→transactions→sanctions→SAR)
2. context_usage: Did agent use memory/context to make informed decisions?
3. trail_following: Did agent properly trace transaction paths?
4. efficiency: Was the investigation path reasonably efficient?
5. score: Overall quality (1-5)

CRITICAL RULES:
- FAILED investigations: max score 2, max strategy_quality 2
- SUCCESS with good strategy: score 4-5
- SUCCESS with poor strategy (got lucky): score 3

Return ONLY valid JSON:
{{"score": X, "strategy_quality": X, "context_usage": X, "trail_following": X, "efficiency": X, "explanation": "brief explanation"}}'''
    
    try:
        # Use new google.genai API
        response = gemini_client.models.generate_content(
            model=GEMINI_MODEL,
            contents=prompt
        )
        json_match = re.search(r'\{[^}]+\}', response.text.replace('\n', ' '))
        if json_match:
            result = json.loads(json_match.group())
            # Enforce caps for failed investigations
            if not episode.success:
                result['score'] = min(result.get('score', 2), 2)
                result['strategy_quality'] = min(result.get('strategy_quality', 2), 2)
            return result
    except Exception as e:
        pass
    
    # Fallback with outcome-based scoring
    base_score = 4 if episode.success else 1
    return {"score": base_score, "strategy_quality": base_score, "context_usage": base_score, 
            "trail_following": base_score, "efficiency": base_score, "explanation": "Gemini parse error - fallback"}

print("✓ Gemini trajectory evaluator defined")


✓ Gemini client initialized: gemini-2.0-flash
✓ Gemini trajectory evaluator defined


In [11]:
def get_investigation_start_accounts(n: int = EVAL_EPISODES) -> List[str]:
    """Get accounts that are starting points of laundering chains."""
    laundering_sources = transactions_df[transactions_df['is_laundering'] == 1]['from_account'].unique()
    laundering_dests = set(transactions_df[transactions_df['is_laundering'] == 1]['to_account'].unique())
    start_accounts = [acc for acc in laundering_sources if acc not in laundering_dests]
    
    if len(start_accounts) < n:
        start_accounts = list(laundering_sources)
    
    random.seed(RANDOM_SEED)
    return random.sample(start_accounts, min(n, len(start_accounts)))

def run_evaluation(model, tokenizer, stage_name: str, n_episodes: int = EVAL_EPISODES, verbose: bool = True) -> dict:
    """Run evaluation using MODEL-DRIVEN episodes (non-deterministic) with separate MLflow experiment."""
    # Create separate MLflow experiment for each stage
    experiment_name = f"AML_Investigation_{stage_name}"
    mlflow.set_experiment(experiment_name)
    
    print(f"\n{'='*70}")
    print(f"🧪 EVALUATION: {stage_name} (MLflow Experiment: {experiment_name})")
    print(f"{'='*70}")
    
    start_accounts = get_investigation_start_accounts(n_episodes)
    episodes = []
    
    with mlflow.start_run(run_name=f"{stage_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        mlflow.log_params({"stage": stage_name, "n_episodes": n_episodes, "max_steps": MAX_STEPS})
        
        start_time = time.time()
        for i, account in enumerate(start_accounts):
            print(f"\n{'─'*60}")
            print(f"  Episode [{i+1}/{n_episodes}]: {account}")
            print(f"{'─'*60}")
            
            # Use MODEL-DRIVEN episode for non-deterministic trajectory
            episode = run_model_driven_episode(model, tokenizer, account, verbose=verbose)
            episodes.append(episode)
        
        total_time = time.time() - start_time
        
        # Compute metrics
        successes = sum(1 for ep in episodes if ep.success)
        success_rate = successes / len(episodes)
        avg_steps = np.mean([len(ep.steps) for ep in episodes])
        terminated_count = sum(1 for ep in episodes if ep.terminated)
        
        # Compute fallback rate - how often model output was unparseable
        total_steps = sum(len(ep.steps) for ep in episodes)
        fallback_steps = sum(
            1 for ep in episodes for step in ep.steps 
            if step.get("used_fallback", False)
        )
        fallback_rate = fallback_steps / total_steps if total_steps > 0 else 0
        
        # Strategy evaluation
        strategy_scores, context_scores, trail_scores, efficiency_scores = [], [], [], []
        for ep in episodes:
            gt_target = env.laundering_targets[0] if env.laundering_targets else "Unknown"
            result = evaluate_investigation_with_gemini(ep, gt_target)
            strategy_scores.append(result.get('strategy_quality', 0))
            context_scores.append(result.get('context_usage', 0))
            trail_scores.append(result.get('trail_following', 0))
            efficiency_scores.append(result.get('efficiency', 0))
        
        avg_strategy = np.mean(strategy_scores)
        avg_context = np.mean(context_scores)
        avg_trail = np.mean(trail_scores)
        avg_efficiency = np.mean(efficiency_scores)
        
        mlflow.log_metrics({
            "success_rate": success_rate, "avg_steps": avg_steps,
            "fallback_rate": fallback_rate,  # Key metric: how often model output was unparseable
            "strategy_quality": avg_strategy, "context_usage": avg_context,
            "trail_following": avg_trail, "efficiency": avg_efficiency
        })
        
        print(f"\n{'='*70}")
        print(f"📊 {stage_name} RESULTS")
        print(f"{'='*70}")
        print(f"  ✅ Success Rate: {success_rate*100:.1f}% ({successes}/{len(episodes)})")
        print(f"  📈 Avg Steps: {avg_steps:.1f}")
        print(f"  🔄 Fallback Rate: {fallback_rate*100:.1f}% ({fallback_steps}/{total_steps} steps)")
        print(f"  📋 Strategy Quality: {avg_strategy:.2f}/5")
        print(f"  🧠 Context Usage: {avg_context:.2f}/5")
        print(f"  🔗 Trail Following: {avg_trail:.2f}/5")
        print(f"  ⚡ Efficiency: {avg_efficiency:.2f}/5")
        print(f"  ⏱️  Time: {total_time:.1f}s")
        print(f"{'='*70}")
        
        return {
            "stage": stage_name, "success_rate": success_rate, "avg_steps": avg_steps,
            "fallback_rate": fallback_rate,  # How often model output was unparseable
            "strategy_quality": avg_strategy, "context_usage": avg_context,
            "trail_following": avg_trail, "efficiency": avg_efficiency,
            "total_time": total_time, "episodes": episodes
        }

all_eval_results = []
print("✓ Evaluation function defined")


✓ Evaluation function defined


## 9. Baseline Evaluation


In [12]:
# Run baseline evaluation
baseline_results = run_evaluation(model, tokenizer, "Baseline", n_episodes=EVAL_EPISODES)
all_eval_results.append(baseline_results)


🧪 EVALUATION: Baseline (MLflow Experiment: AML_Investigation_Baseline)

────────────────────────────────────────────────────────────
  Episode [1/10]: 136009-80D626250
────────────────────────────────────────────────────────────

🔍 LANGGRAPH INVESTIGATION: 136009-80D626250
   Memory managed by: LangGraph MemorySaver


2025/12/28 13:29:49 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: Yaml file './mlruns/782179418682474946/traces/tr-8b9d2434e465e150bd9c66b3ad3c2d6d/trace_info.yaml' exists as './mlruns/782179418682474946/traces/tr-8b9d2434e465e150bd9c66b3ad3c2d6d/trace_info.yaml



┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking> I need to gather information about account 136009-80D626250 first.  I'll start with the a...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 
Strategy: EXPLORE
│ 🔧 TOOL: get_account_summary()
│ 📤 RESULT: {"error": "Missing account_id argument"}...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>The tool is failing to process the argument.  I need to ensure I have the correct account ...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 
Strategy: EXPLORE
│ 🔧 TOOL: get_account_id(account_id=136009-80D626250)
│ 📤 RESULT: {"error": "Unknown tool: get_account_id"}...

┌─ Step 3 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking> I'm having trouble finding the account ID. I need to double-check the information provide...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 
Strategy:

2025/12/28 13:31:25 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: Yaml file './mlruns/782179418682474946/traces/tr-827050a82369b584ff5e9ff0ff50bde4/trace_info.yaml' exists as './mlruns/782179418682474946/traces/tr-827050a82369b584ff5e9ff0ff50bde4/trace_info.yaml



┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>I need to get the initial information about account 11-800B654E0 to understand its risk an...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 11-800B654E0
Strategy: EXPLORE
│ 🔧 TOOL: get_account_summary(account_id=11-800B654E0)
│ 📤 RESULT: {"account_id": "11-800B654E0", "account_type": "Sole Proprietorship", "entity_na...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>The account is a Sole Proprietorship with a relatively low risk score. I should investigat...
│ 🧠 MEMORY (LangGraph state): Progress: 5 accounts | $16,969 traced
Path: 11314-814D8DE50 → 0-800B65530 → 4308
│ 🔧 TOOL: get_recent_transactions(account_id=11-800B654E0)
│ 📤 RESULT: [{"to_account": "11-800B654E0", "amount": 6866.15, "currency": "US Dollar"}, {"t...

┌─ Step 3 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>The account received significan

## 10. Pattern-Aware SFT Training Dataset

### Strategy Overview

The IBM AML dataset includes **pattern files** that document the exact structure of each laundering attempt. 
We leverage this ground truth to generate high-quality training samples that teach:

1. **Pattern Recognition**: Each laundering scheme has a specific structure (FAN-IN, FAN-OUT, CYCLE, etc.)
2. **Tool Sequencing**: The correct order of tool calls for effective investigation
3. **Strategy Adaptation**: Different investigation strategies for different pattern types
4. **Multi-hop Following**: Following money trails through multiple intermediaries

### Laundering Pattern Types

| Pattern | Structure | Investigation Strategy |
|---------|-----------|------------------------|
| **FAN-IN** | Many → One | Check the hub receiving funds for sanctions |
| **FAN-OUT** | One → Many | Follow all outgoing transactions from source |
| **GATHER-SCATTER** | Many → Hub → Many | Trace through the central hub |
| **SCATTER-GATHER** | One → Many → One | Follow scattered funds to convergence point |
| **CYCLE** | A → ... → A | Detect circular flow (same account appears twice) |
| **STACK** | Multiple parallel chains | Investigate independent 2-hop chains |
| **RANDOM** | Variable-length chain | Follow hop-by-hop until sanctioned entity |
| **BIPARTITE** | Two-group cross-connections | Map connections between account groups |

### Training Sample Format

Each sample demonstrates a complete investigation with:
- Memory context showing investigation progress
- Appropriate tool calls based on pattern type
- Strategy reasoning in the model's response
- Final SAR submission with pattern-specific justification


In [13]:
# =============================================================================
# PATTERN FILE PARSER
# =============================================================================
# The IBM AML dataset includes pattern files (e.g., LI-Small_Patterns.txt) that
# document the exact structure of each money laundering attempt. Each pattern
# is bracketed by BEGIN/END markers and contains the transaction sequence.
#
# Example pattern structure:
#   BEGIN LAUNDERING ATTEMPT - FAN-IN: Max 3-degree Fan-In
#   2022/09/01 02:38,001812,80279F810,0110,8000A94C0,10154.74,Australian Dollar,...
#   2022/09/02 14:36,022595,80279F8B0,0110,8000A94C0,5326.79,Australian Dollar,...
#   END LAUNDERING ATTEMPT - FAN-IN
# =============================================================================

def parse_patterns_file(filepath: str) -> List[dict]:
    """
    Parse IBM AML patterns file into structured laundering attempts.
    
    Args:
        filepath: Path to the patterns file (e.g., LI-Small_Patterns.txt)
    
    Returns:
        List of pattern dictionaries, each containing:
        - type: Pattern type (FAN-IN, FAN-OUT, CYCLE, etc.)
        - details: Additional info (e.g., "Max 3-degree Fan-In")
        - transactions: List of transaction dicts with from/to accounts and amounts
    
    Pattern Types in IBM AML Dataset:
    - FAN-IN: Multiple accounts send to one hub (classic structuring)
    - FAN-OUT: One account distributes to many (layering)
    - GATHER-SCATTER: Many → Hub → Many (integration pattern)
    - SCATTER-GATHER: One → Many → One (diamond pattern)
    - CYCLE: Money flows back to origin (circular laundering)
    - STACK: Multiple parallel 2-hop chains (obfuscation)
    - RANDOM: Variable-length random path (noise pattern)
    - BIPARTITE: Two groups with cross-connections
    """
    patterns = []
    current_pattern = None
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Detect pattern start
            if line.startswith("BEGIN LAUNDERING ATTEMPT"):
                # Parse: "BEGIN LAUNDERING ATTEMPT - FAN-IN: Max 3-degree Fan-In"
                parts = line.split(" - ")
                if len(parts) >= 2:
                    type_and_details = parts[1]
                    if ":" in type_and_details:
                        pattern_type, details = type_and_details.split(":", 1)
                        pattern_type = pattern_type.strip()
                        details = details.strip()
                    else:
                        pattern_type = type_and_details.strip()
                        details = ""
                else:
                    pattern_type = "UNKNOWN"
                    details = ""
                
                current_pattern = {
                    "type": pattern_type,
                    "details": details,
                    "transactions": []
                }
            
            # Detect pattern end
            elif line.startswith("END LAUNDERING ATTEMPT"):
                if current_pattern and len(current_pattern["transactions"]) > 0:
                    patterns.append(current_pattern)
                current_pattern = None
            
            # Parse transaction line
            elif current_pattern and "," in line:
                # Format: timestamp,from_bank,from_acc,to_bank,to_acc,amount,currency,...
                parts = line.split(",")
                if len(parts) >= 10:
                    try:
                        txn = {
                            "timestamp": parts[0],
                            "from_account": f"{parts[1]}-{parts[2]}",  # bank-account format
                            "to_account": f"{parts[3]}-{parts[4]}",
                            "amount": float(parts[5]),
                            "currency": parts[6],
                        }
                        current_pattern["transactions"].append(txn)
                    except (ValueError, IndexError):
                        continue  # Skip malformed lines
    
    return patterns

# =============================================================================
# INVESTIGATION STRATEGY BY PATTERN TYPE
# =============================================================================
# Each pattern type requires a different investigation approach. These strategies
# are embedded in training samples to teach the model pattern-specific behavior.
# =============================================================================

PATTERN_STRATEGIES = {
    "FAN-IN": {
        "description": "Multiple accounts consolidating into one hub",
        "strategy": "Identify the receiving hub, check for sanctions, trace sources",
        "key_insight": "The hub receiving funds is the primary target",
        "tool_priority": ["get_account_summary", "get_recent_transactions", "check_sanctions_list"],
    },
    "FAN-OUT": {
        "description": "One account distributing to many recipients",
        "strategy": "Follow all outgoing transactions, check each recipient",
        "key_insight": "The source is laundering, recipients may be shells",
        "tool_priority": ["get_recent_transactions", "get_account_summary", "check_sanctions_list"],
    },
    "GATHER-SCATTER": {
        "description": "Many sources → Hub → Many destinations",
        "strategy": "Trace through the central hub to find endpoints",
        "key_insight": "The hub is the critical chokepoint",
        "tool_priority": ["get_recent_transactions", "check_sanctions_list", "get_account_summary"],
    },
    "SCATTER-GATHER": {
        "description": "One source → Many intermediaries → One destination",
        "strategy": "Follow scattered funds to convergence point",
        "key_insight": "The final destination is the target",
        "tool_priority": ["get_recent_transactions", "get_account_summary", "check_sanctions_list"],
    },
    "CYCLE": {
        "description": "Circular money flow returning to origin",
        "strategy": "Detect when an account appears multiple times in the path",
        "key_insight": "Circular flow is a strong laundering indicator",
        "tool_priority": ["get_recent_transactions", "get_account_summary", "check_sanctions_list"],
    },
    "STACK": {
        "description": "Multiple independent 2-hop chains",
        "strategy": "Investigate each chain independently",
        "key_insight": "Parallel chains obscure the total volume",
        "tool_priority": ["get_account_summary", "get_recent_transactions", "check_sanctions_list"],
    },
    "RANDOM": {
        "description": "Variable-length chain with random hops",
        "strategy": "Follow the chain hop-by-hop until endpoint",
        "key_insight": "Each hop may have different amounts/currencies",
        "tool_priority": ["get_recent_transactions", "get_account_summary", "check_sanctions_list"],
    },
    "BIPARTITE": {
        "description": "Two groups of accounts with cross-connections",
        "strategy": "Map the connections between account groups",
        "key_insight": "Cross-group transactions indicate coordination",
        "tool_priority": ["get_recent_transactions", "get_account_summary", "check_sanctions_list"],
    },
}

# =============================================================================
# PATTERN-AWARE SFT DATASET GENERATOR
# =============================================================================
# This generator creates training samples that teach the model:
# 1. How to recognize different laundering patterns
# 2. The correct tool calling sequence for each pattern
# 3. How to adapt strategy based on investigation progress
# 4. When and how to submit a SAR with pattern-specific justification
# =============================================================================

def generate_pattern_aware_sft_dataset(patterns: List[dict], n_samples: int = 200) -> List[dict]:
    """
    Generate SFT training samples from parsed laundering patterns.
    
    Each sample demonstrates a complete investigation with:
    - Progressive memory context updates
    - Pattern-appropriate tool calls
    - Strategy reasoning in responses
    - Final SAR with pattern-specific justification
    
    Args:
        patterns: List of parsed patterns from parse_patterns_file()
        n_samples: Maximum number of samples to generate
    
    Returns:
        List of training samples with 'messages' and metadata
    """
    samples = []
    
    # Get sanctioned accounts for realistic responses
    sanctioned_set = set(accounts_pl.filter(pl.col('is_sanctioned') == True)['account_id'].to_list())
    
    for pattern in patterns[:n_samples]:
        pattern_type = pattern["type"]
        txns = pattern["transactions"]
        
        # Skip patterns with too few transactions
        if len(txns) < 2:
            continue
        
        # Get strategy for this pattern type
        strategy_info = PATTERN_STRATEGIES.get(pattern_type, PATTERN_STRATEGIES["RANDOM"])
        
        # Build the account flow graph for this pattern
        accounts_in_pattern = []
        for txn in txns:
            if txn["from_account"] not in accounts_in_pattern:
                accounts_in_pattern.append(txn["from_account"])
            if txn["to_account"] not in accounts_in_pattern:
                accounts_in_pattern.append(txn["to_account"])
        
        # Identify the investigation target based on pattern type
        if pattern_type == "FAN-IN":
            # Target is the receiving hub (most common destination)
            dest_counts = {}
            for txn in txns:
                dest_counts[txn["to_account"]] = dest_counts.get(txn["to_account"], 0) + 1
            target = max(dest_counts, key=dest_counts.get)
            start = txns[0]["from_account"]
        elif pattern_type == "FAN-OUT":
            # Start from the source (most common origin)
            src_counts = {}
            for txn in txns:
                src_counts[txn["from_account"]] = src_counts.get(txn["from_account"], 0) + 1
            start = max(src_counts, key=src_counts.get)
            target = txns[-1]["to_account"]
        elif pattern_type == "CYCLE":
            # Start and end are the same account
            start = txns[0]["from_account"]
            target = txns[-1]["to_account"]
        else:
            # Default: start from first, target is last
            start = txns[0]["from_account"]
            target = txns[-1]["to_account"]
        
        # Calculate total amount in pattern
        total_amount = sum(txn["amount"] for txn in txns)
        
        # Build training conversation
        # =========================================================================
        # System message explains the pattern context
        # =========================================================================
        system_msg = f"""You are an AML investigation agent. You have detected a potential {pattern_type} laundering pattern.

Pattern Description: {strategy_info['description']}
Investigation Strategy: {strategy_info['strategy']}
Key Insight: {strategy_info['key_insight']}

Use the available tools to investigate and submit a SAR when you find a sanctioned entity.

TOOLS:
1. get_account_summary(account_id) - Get account metadata and risk score
2. get_recent_transactions(account_id, direction) - Get transactions (direction: "outgoing" or "incoming")
3. check_sanctions_list(entity_id) - Check if entity is on sanctions list
4. submit_suspicious_activity_report(entity_id, reason) - Submit SAR (terminal action)

RESPONSE FORMAT:
ACTION: tool_name
PARAMS: {{"param1": "value1"}}"""
        
        messages = [{"role": "system", "content": system_msg}]
        
        # =========================================================================
        # Progressive investigation steps
        # =========================================================================
        mem_state = {
            "accounts_analyzed": 0,
            "entities_checked": 0,
            "amount_traced": 0.0,
            "pattern_detected": pattern_type,
            "strategy": "explore"
        }
        
        # Limit steps to avoid overly long sequences
        max_steps = min(len(txns), 8)
        
        for step_idx, txn in enumerate(txns[:max_steps]):
            from_acc = txn["from_account"]
            to_acc = txn["to_account"]
            amount = txn["amount"]
            
            mem_state["amount_traced"] += amount
            
            # Update strategy based on progress
            if mem_state["amount_traced"] > 100000:
                mem_state["strategy"] = "follow_high_value"
            elif mem_state["accounts_analyzed"] > 3:
                mem_state["strategy"] = "trace_chain"
            
            # =====================================================================
            # Step A: Memory context prompt
            # =====================================================================
            memory_context = f"""INVESTIGATION MEMORY:
- Pattern: {pattern_type}
- Accounts analyzed: {mem_state['accounts_analyzed']}
- Entities checked for sanctions: {mem_state['entities_checked']}
- Amount traced: ${mem_state['amount_traced']:,.2f}
- Current strategy: {mem_state['strategy'].upper()}

Current focus: {from_acc} → {to_acc} (${amount:,.2f})"""
            
            messages.append({"role": "user", "content": memory_context})
            
            # =====================================================================
            # Step B: Get account summary (if not already analyzed)
            # =====================================================================
            if step_idx == 0 or random.random() > 0.5:  # Add variety
                mem_state["accounts_analyzed"] += 1
                
                # Simulate account type
                acc_type = random.choice(["Corporate", "Partnership", "Sole Proprietorship"])
                risk = round(random.uniform(0.4, 0.9), 2)
                
                assistant_response = f"""Based on the {pattern_type} pattern, I'll check the account details.

ACTION: get_account_summary
PARAMS: {{"account_id": "{from_acc}"}}"""
                
                messages.append({"role": "assistant", "content": assistant_response})
                
                tool_result = json.dumps({
                    "account_id": from_acc,
                    "account_type": acc_type,
                    "risk_score": risk,
                    "is_sanctioned": from_acc in sanctioned_set
                })
                messages.append({"role": "user", "content": f"TOOL RESULT: {tool_result}"})
            
            # =====================================================================
            # Step C: Get transactions
            # =====================================================================
            direction = "outgoing" if pattern_type in ["FAN-OUT", "SCATTER-GATHER"] else "outgoing"
            
            assistant_response = f"""Following the money trail in this {pattern_type} pattern.

ACTION: get_recent_transactions
PARAMS: {{"account_id": "{from_acc}", "direction": "{direction}"}}"""
            
            messages.append({"role": "assistant", "content": assistant_response})
            
            txn_result = json.dumps([{
                "to_account": to_acc,
                "amount": amount,
                "currency": txn["currency"]
            }])
            messages.append({"role": "user", "content": f"TOOL RESULT: {txn_result}"})
            
            # =====================================================================
            # Step D: Check sanctions (especially for later steps)
            # =====================================================================
            if step_idx >= len(txns[:max_steps]) - 2:  # Last couple of steps
                mem_state["entities_checked"] += 1
                is_sanctioned = to_acc in sanctioned_set or step_idx == len(txns[:max_steps]) - 1
                
                assistant_response = f"""Checking if the recipient is on sanctions list.

ACTION: check_sanctions_list
PARAMS: {{"entity_id": "{to_acc}"}}"""
                
                messages.append({"role": "assistant", "content": assistant_response})
                
                sanctions_result = json.dumps({
                    "entity_id": to_acc,
                    "on_sanctions_list": is_sanctioned,
                    "list_type": "OFAC SDN" if is_sanctioned else None
                })
                messages.append({"role": "user", "content": f"TOOL RESULT: {sanctions_result}"})
        
        # =========================================================================
        # Final SAR submission
        # =========================================================================
        final_reason = f"{pattern_type} pattern detected - {strategy_info['key_insight']}. Total amount: ${total_amount:,.2f}"
        
        final_response = f"""SANCTIONED ENTITY FOUND! Submitting Suspicious Activity Report.

This investigation revealed a {pattern_type} laundering pattern:
- {strategy_info['description']}
- Total amount traced: ${total_amount:,.2f}
- {len(accounts_in_pattern)} accounts involved

ACTION: submit_suspicious_activity_report
PARAMS: {{"entity_id": "{target}", "reason": "{final_reason}"}}"""
        
        messages.append({"role": "assistant", "content": final_response})
        
        # Add sample to dataset
        samples.append({
            "messages": messages,
            "pattern_type": pattern_type,
            "pattern_details": pattern["details"],
            "num_transactions": len(txns),
            "total_amount": total_amount,
            "target_account": target,
            "accounts_involved": len(accounts_in_pattern),
        })
    
        # =====================================================================
        # SAMPLE 7: ALL entities checked, NONE sanctioned - DO NOT SUBMIT SAR
        # This teaches the model to STOP when no sanctioned entity is found
        # =====================================================================
        if len(accounts_in_pattern) >= 2:
            checked_1 = accounts_in_pattern[0]
            checked_2 = accounts_in_pattern[1]
            
            memory_7 = f"Progress: 4 accounts | ${total_amount:,.0f} traced\nPath: {' → '.join(accounts_in_pattern[:3])}\nRisks: HIGH_RISK_SCORE\nStrategy: EXPLORE"
            
            prompt_7 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_7}

PREVIOUS RESULT: Checked {checked_1} - NOT on sanctions list. Checked {checked_2} - NOT on sanctions list.<end_of_turn>
<start_of_turn>model
<thinking>Multiple entities checked but none sanctioned. Cannot submit SAR without confirmed sanctioned entity. Need to trace more transactions.</thinking>
ACTION: get_recent_transactions
PARAMS: {{"account_id": "{checked_2}", "direction": "outgoing"}}<end_of_turn>"""
            
            samples.append({"text": prompt_7, "step": "continue_no_sanctions", "pattern": pattern_type})
    
    return samples


# =============================================================================
# INFERENCE-MATCHING SFT DATASET GENERATOR
# =============================================================================
# CRITICAL: Training data must match EXACTLY what build_prompt_from_state produces.
# This ensures the model learns the correct format for tool calling.
#
# Inference prompt format:
#   <start_of_turn>user
#   Investigate account {account} for money laundering.
#   {TOOL_DESC}
#   CURRENT STATUS:
#   {memory_context}<end_of_turn>
#   <start_of_turn>model
#   {reasoning and tool call}<end_of_turn>
# =============================================================================

# The TOOL_DESC used during inference (must match exactly)
# SFT_TOOL_DESC matches inference TOOL_DESC exactly (with thinking)
SFT_TOOL_DESC = """You are an AML investigator. Use these tools to investigate money laundering:

TOOLS:
- get_account_summary(account_id): Get account info and risk score
- get_recent_transactions(account_id, direction): Get transactions (direction: "outgoing" or "incoming")
- check_sanctions_list(entity_id): Check if entity is on sanctions list
- submit_suspicious_activity_report(entity_id, reason): Submit SAR (ONLY if entity is confirmed sanctioned!)

RESPONSE FORMAT:
<thinking>Brief reasoning about your next action</thinking>
ACTION: tool_name
PARAMS: {"key": "value"}

RULES:
- Always use <thinking> before ACTION to explain your reasoning
- NEVER submit SAR unless check_sanctions_list confirmed the entity is sanctioned
- Follow money trails through transactions to find suspicious entities"""


def generate_inference_matching_sft_dataset(patterns: List[dict], n_samples: int = 200) -> List[dict]:
    """
    Generate SFT samples that EXACTLY match the inference prompt format.
    
    CRITICAL: Each sample has only ONE model response (the target action).
    The memory context from LangGraph MemorySaver is captured in CURRENT STATUS.
    
    Structure for each sample:
    - User message with investigation task + tools + CURRENT STATUS (memory context)
    - Previous tool results as user messages (if any)
    - Single <start_of_turn>model with the TARGET action only
    
    This ensures the model learns:
    1. Given memory context → Select appropriate tool
    2. Given tool results → Decide next action
    3. Given sanctions check result → Submit SAR or continue
    """
    samples = []
    sanctioned_set = set(accounts_pl.filter(pl.col('is_sanctioned') == True)['account_id'].to_list())
    
    for pattern in patterns[:n_samples]:
        pattern_type = pattern["type"]
        txns = pattern["transactions"]
        
        if len(txns) < 2:
            continue
        
        # Get strategy info
        strategy_info = PATTERN_STRATEGIES.get(pattern_type, PATTERN_STRATEGIES["RANDOM"])
        
        # Build account list for this pattern
        accounts_in_pattern = []
        for txn in txns:
            if txn["from_account"] not in accounts_in_pattern:
                accounts_in_pattern.append(txn["from_account"])
            if txn["to_account"] not in accounts_in_pattern:
                accounts_in_pattern.append(txn["to_account"])
        
        start_account = txns[0]["from_account"]
        target_account = txns[-1]["to_account"]
        total_amount = sum(t["amount"] for t in txns)
        
        # =====================================================================
        # SAMPLE 1: Initial step - get_account_summary
        # Memory shows fresh start, model should get account summary
        # =====================================================================
        memory_1 = "Progress: 0 accounts | $0 traced\nPath: (starting)\nStrategy: EXPLORE"
        
        prompt_1 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_1}<end_of_turn>
<start_of_turn>model
<thinking>Starting investigation. First step is to get account summary to assess risk level.</thinking>
ACTION: get_account_summary
PARAMS: {{"account_id": "{start_account}"}}<end_of_turn>"""
        
        samples.append({"text": prompt_1, "step": "get_account_summary", "pattern": pattern_type})
        
        # =====================================================================
        # SAMPLE 2: After account summary - get_recent_transactions
        # Memory shows 1 account analyzed, model should get transactions
        # Tool result is provided as context, model outputs ONLY the new action
        # =====================================================================
        memory_2 = f"Progress: 1 accounts | $0 traced\nPath: {start_account}\nRisks: HIGH_RISK_SCORE\nStrategy: EXPLORE"
        
        prompt_2 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_2}

PREVIOUS RESULT: Account {start_account} has risk_score=85, account_type=Corporate<end_of_turn>
<start_of_turn>model
<thinking>Account has high risk score (85). Need to trace money flow by checking outgoing transactions.</thinking>
ACTION: get_recent_transactions
PARAMS: {{"account_id": "{start_account}", "direction": "outgoing"}}<end_of_turn>"""
        
        samples.append({"text": prompt_2, "step": "get_recent_transactions", "pattern": pattern_type})
        
        # =====================================================================
        # SAMPLE 3: After transactions - check_sanctions_list
        # Memory shows money traced, model should check sanctions
        # =====================================================================
        if len(txns) >= 1:
            next_account = txns[0]["to_account"]
            amount = txns[0]["amount"]
            
            memory_3 = f"Progress: 2 accounts | ${amount:,.0f} traced\nPath: {start_account} → {next_account}\nRisks: HIGH_RISK_SCORE; LARGE_TRANSACTION\nStrategy: FOLLOW_MONEY"
            
            prompt_3 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_3}

PREVIOUS RESULT: Found transaction to {next_account} for ${amount:,.2f}<end_of_turn>
<start_of_turn>model
<thinking>Large transaction detected to {next_account}. Must verify if recipient is on sanctions list before proceeding.</thinking>
ACTION: check_sanctions_list
PARAMS: {{"entity_id": "{next_account}"}}<end_of_turn>"""
            
            samples.append({"text": prompt_3, "step": "check_sanctions_list", "pattern": pattern_type})
        
        # =====================================================================
        # SAMPLE 4: Sanctions check returned TRUE - submit SAR
        # Memory shows sanctioned entity found, model should submit SAR
        # =====================================================================
        memory_4 = f"Progress: 3 accounts | ${total_amount:,.0f} traced\nPath: {start_account} → {target_account}\nRisks: HIGH_RISK_SCORE; SANCTIONED_ENTITY_FOUND\nStrategy: REPORT"
        reason = f"{pattern_type} pattern detected - {strategy_info['key_insight']}"
        
        prompt_4 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_4}

PREVIOUS RESULT: Entity {target_account} is ON SANCTIONS LIST (OFAC SDN)<end_of_turn>
<start_of_turn>model
<thinking>Entity {target_account} is confirmed on sanctions list. This confirms money laundering - must submit SAR immediately.</thinking>
ACTION: submit_suspicious_activity_report
PARAMS: {{"entity_id": "{target_account}", "reason": "{reason}"}}<end_of_turn>"""
        
        samples.append({"text": prompt_4, "step": "submit_sar", "pattern": pattern_type})
        
        # =====================================================================
        # SAMPLE 5: Sanctions check returned FALSE - continue investigating
        # Memory shows entity checked but not sanctioned, model should continue
        # =====================================================================
        if len(accounts_in_pattern) >= 3:
            checked_account = accounts_in_pattern[1]
            next_to_check = accounts_in_pattern[2]
            
            memory_5 = f"Progress: 3 accounts | ${txns[0]['amount']:,.0f} traced\nPath: {start_account} → {checked_account}\nRisks: HIGH_RISK_SCORE\nStrategy: FOLLOW_MONEY"
            
            prompt_5 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_5}

PREVIOUS RESULT: Entity {checked_account} is NOT on sanctions list<end_of_turn>
<start_of_turn>model
<thinking>Entity {checked_account} is not sanctioned. Must continue tracing - checking next entity in the chain.</thinking>
ACTION: check_sanctions_list
PARAMS: {{"entity_id": "{next_to_check}"}}<end_of_turn>"""
            
            samples.append({"text": prompt_5, "step": "continue_investigating", "pattern": pattern_type})
        
        # =====================================================================
        # SAMPLE 6: Use ACTUAL sanctioned entity for realistic training
        # =====================================================================
        sanctioned_in_pattern = [acc for acc in accounts_in_pattern if acc in sanctioned_set]
        if sanctioned_in_pattern:
            actual_sanctioned = sanctioned_in_pattern[0]
            
            memory_6 = f"Progress: 4 accounts | ${total_amount:,.0f} traced\nPath: {start_account} → {actual_sanctioned}\nRisks: SANCTIONED_ENTITY_FOUND\nStrategy: REPORT"
            
            prompt_6 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_6}

PREVIOUS RESULT: Entity {actual_sanctioned} is ON SANCTIONS LIST (OFAC SDN)<end_of_turn>
<start_of_turn>model
<thinking>Entity {actual_sanctioned} confirmed on OFAC SDN sanctions list. Evidence is clear - submitting SAR.</thinking>
ACTION: submit_suspicious_activity_report
PARAMS: {{"entity_id": "{actual_sanctioned}", "reason": "Sanctioned entity on OFAC SDN list detected in {pattern_type} money laundering pattern"}}<end_of_turn>"""
            
            samples.append({"text": prompt_6, "step": "submit_sar_real", "pattern": pattern_type})
    
        # =====================================================================
        # SAMPLE 7: ALL entities checked, NONE sanctioned - DO NOT SUBMIT SAR
        # This teaches the model to STOP when no sanctioned entity is found
        # =====================================================================
        if len(accounts_in_pattern) >= 2:
            checked_1 = accounts_in_pattern[0]
            checked_2 = accounts_in_pattern[1]
            
            memory_7 = f"Progress: 4 accounts | ${total_amount:,.0f} traced\nPath: {' → '.join(accounts_in_pattern[:3])}\nRisks: HIGH_RISK_SCORE\nStrategy: EXPLORE"
            
            prompt_7 = f"""<start_of_turn>user
Investigate account {start_account} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
{memory_7}

PREVIOUS RESULT: Checked {checked_1} - NOT on sanctions list. Checked {checked_2} - NOT on sanctions list.<end_of_turn>
<start_of_turn>model
<thinking>Multiple entities checked but none sanctioned. Cannot submit SAR without confirmed sanctioned entity. Need to trace more transactions.</thinking>
ACTION: get_recent_transactions
PARAMS: {{"account_id": "{checked_2}", "direction": "outgoing"}}<end_of_turn>"""
            
            samples.append({"text": prompt_7, "step": "continue_no_sanctions", "pattern": pattern_type})
    
    return samples


# =============================================================================
# LOAD PATTERNS AND GENERATE DATASET
# =============================================================================

# Determine which pattern file to use based on DATASET_SIZE
# Pattern file naming: {DATASET_PREFIX}-{DATASET_SIZE}_Patterns.txt (e.g., LI-Small_Patterns.txt)
pattern_file = os.path.join(DATA_DIR, f"{DATASET_PREFIX}-{DATASET_SIZE}_Patterns.txt")

if os.path.exists(pattern_file):
    print(f"📂 Loading patterns from: {pattern_file}")
    laundering_patterns = parse_patterns_file(pattern_file)
    print(f"  ✓ Parsed {len(laundering_patterns)} laundering patterns")
    
    # Show pattern type distribution
    pattern_types = {}
    for p in laundering_patterns:
        pt = p["type"]
        pattern_types[pt] = pattern_types.get(pt, 0) + 1
    
    print(f"\n  Pattern Distribution:")
    for pt, count in sorted(pattern_types.items(), key=lambda x: -x[1]):
        print(f"    - {pt}: {count}")
    
    # Generate INFERENCE-MATCHING SFT dataset
    print(f"\n🎯 Generating inference-matching SFT dataset...")
    sft_dataset = generate_inference_matching_sft_dataset(laundering_patterns, n_samples=200)
    print(f"  ✓ Generated {len(sft_dataset)} training samples")
    
    # Show step distribution
    step_counts = {}
    for s in sft_dataset:
        step = s.get("step", "unknown")
        step_counts[step] = step_counts.get(step, 0) + 1
    
    print(f"\n  Step Distribution:")
    for step, count in step_counts.items():
        print(f"    - {step}: {count}")
else:
    print(f"⚠️ Pattern file not found: {pattern_file}")
    print("  Generating simple training samples from laundering transactions...")
    
    # Simple fallback using transaction data
    sft_dataset = []
    laundering_txns = transactions_pl.filter(pl.col('is_laundering') == 1).head(200).to_dicts()
    
    for txn in laundering_txns:
        start = txn["from_account"]
        target = txn["to_account"]
        
        # Simple get_account_summary sample
        prompt = f"""<start_of_turn>user
Investigate account {start} for money laundering.

{SFT_TOOL_DESC}

CURRENT STATUS:
Progress: 0 accounts | $0 traced
Strategy: EXPLORE<end_of_turn>
<start_of_turn>model
I'll start by getting the account summary.

ACTION: get_account_summary
PARAMS: {{"account_id": "{start}"}}<end_of_turn>"""
        sft_dataset.append({"text": prompt, "step": "get_account_summary"})
    
    print(f"  ✓ Generated {len(sft_dataset)} fallback samples")

print(f"\n✅ SFT dataset ready: {len(sft_dataset)} samples")

# =============================================================================
# SAMPLE VALIDATION AND INSPECTION
# =============================================================================
# This is crucial for debugging - ensure training data has correct format

if sft_dataset:
    print(f"\n{'='*70}")
    print("🔍 SFT DATASET VALIDATION")
    print(f"{'='*70}")
    
    # Validate all samples have correct format
    valid_samples = 0
    format_issues = []
    
    for i, sample in enumerate(sft_dataset):
        text = sample.get("text", "")
        has_user_turn = "<start_of_turn>user" in text
        has_model_turn = "<start_of_turn>model" in text
        has_action = "ACTION:" in text
        has_params = "PARAMS:" in text
        has_end = "<end_of_turn>" in text
        
        if all([has_user_turn, has_model_turn, has_action, has_params, has_end]):
            valid_samples += 1
        else:
            if len(format_issues) < 5:  # Only record first 5 issues
                issues = []
                if not has_user_turn: issues.append("missing <start_of_turn>user")
                if not has_model_turn: issues.append("missing <start_of_turn>model")
                if not has_action: issues.append("missing ACTION:")
                if not has_params: issues.append("missing PARAMS:")
                if not has_end: issues.append("missing <end_of_turn>")
                format_issues.append((i, issues))
    
    print(f"✓ Valid samples: {valid_samples}/{len(sft_dataset)} ({100*valid_samples/len(sft_dataset):.1f}%)")
    
    if format_issues:
        print(f"⚠️ Format issues found:")
        for idx, issues in format_issues:
            print(f"   Sample {idx}: {', '.join(issues)}")
    
    # Show average sample length
    avg_len = sum(len(s["text"]) for s in sft_dataset) / len(sft_dataset)
    print(f"📏 Average sample length: {avg_len:.0f} chars")
    
    # Show sample previews (different step types)
    print(f"\n{'='*70}")
    print("📋 SAMPLE PREVIEWS")
    print(f"{'='*70}")
    
    # Find one sample of each step type
    step_types = ["get_account_summary", "get_recent_transactions", "check_sanctions_list", "submit_sar"]
    shown_steps = set()
    
    for sample in sft_dataset:
        step = sample.get("step", "unknown")
        if step not in shown_steps and len(shown_steps) < 3:
            shown_steps.add(step)
            print(f"\n--- [{step}] ---")
            # Show model response part only (after <start_of_turn>model)
            text = sample["text"]
            model_start = text.find("<start_of_turn>model")
            if model_start != -1:
                model_response = text[model_start:][:300]
                print(model_response + "..." if len(text[model_start:]) > 300 else model_response)
            else:
                print(text[:300] + "...")
    
    print(f"\n{'='*70}")


📂 Loading patterns from: /workspace/data/raw/LI-Small_Patterns.txt
  ✓ Parsed 117 laundering patterns

  Pattern Distribution:
    - FAN-OUT: 19
    - STACK: 18
    - BIPARTITE: 16
    - RANDOM: 15
    - SCATTER-GATHER: 13
    - FAN-IN: 12
    - GATHER-SCATTER: 12
    - CYCLE: 12

🎯 Generating inference-matching SFT dataset...
  ✓ Generated 639 training samples

  Step Distribution:
    - get_account_summary: 109
    - get_recent_transactions: 109
    - check_sanctions_list: 109
    - submit_sar: 109
    - continue_investigating: 94
    - continue_no_sanctions: 109

✅ SFT dataset ready: 639 samples

🔍 SFT DATASET VALIDATION
✓ Valid samples: 639/639 (100.0%)
📏 Average sample length: 1338 chars

📋 SAMPLE PREVIEWS

--- [get_account_summary] ---
<start_of_turn>model
<thinking>Starting investigation. First step is to get account summary to assess risk level.</thinking>
ACTION: get_account_summary
PARAMS: {"account_id": "001812-80279F810"}<end_of_turn>

--- [get_recent_transactions] ---
<sta

## 11. Supervised Fine-Tuning with Unsloth

### Key Improvements

1. **Automatic Cleanup**: Previous training artifacts (`sft_output/`, `sft_adapter/`) are removed before each run
2. **Inference-Matching Format**: Training data uses the EXACT same format as `build_prompt_from_state`
3. **Increased Training**: 3 epochs with cosine LR decay and warmup
4. **Better LoRA Config**: Higher alpha (32) and small dropout for regularization
5. **Optimized Hyperparameters**: Lower learning rate (5e-5) for stable convergence


In [14]:
# =============================================================================
# SUPERVISED FINE-TUNING WITH CLEANUP
# =============================================================================
# Clean up previous run artifacts to ensure fresh training
# =============================================================================

import shutil
import gc

# Clean up previous SFT outputs
SFT_OUTPUT_DIR = "./sft_output"
SFT_ADAPTER_DIR = "./sft_adapter"

for cleanup_dir in [SFT_OUTPUT_DIR, SFT_ADAPTER_DIR]:
    if os.path.exists(cleanup_dir):
        shutil.rmtree(cleanup_dir, ignore_errors=True)
        print(f"🧹 Cleaned up: {cleanup_dir}")

print("✓ Previous SFT artifacts cleaned")

# =============================================================================
# RELOAD BASE MODEL IF NEEDED
# =============================================================================
# If model already has LoRA adapters from a previous run, we need to reload
# the base model to start fresh. This ensures consistent training each run.

from trl import SFTTrainer, SFTConfig
from datasets import Dataset
from unsloth import FastLanguageModel

# Check if model already has LoRA adapters
has_existing_adapters = hasattr(model, 'peft_config') or hasattr(model, 'active_adapter')

if has_existing_adapters:
    print("⚠️ Model has existing LoRA adapters - reloading base model...")
    
    # Clear GPU memory
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    # Reload base model fresh
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=16384,  # Match initial loading for consistency
        dtype=None,
        load_in_4bit=True,
    )
    print("✓ Base model reloaded")
    
    # Update global references for the agent
    set_model_for_agent(model, tokenizer)

# =============================================================================
# PREPARE MODEL FOR TRAINING
# =============================================================================

# Prepare model for training with Unsloth's optimized PEFT
# LoRA config: r=16 is a good balance between quality and efficiency
model = FastLanguageModel.get_peft_model(
    model, 
    r=16,  # LoRA rank - higher = more capacity but slower
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,  # Increased from 16 for stronger updates
    lora_dropout=0.05,  # Small dropout for regularization
    bias="none", 
    use_gradient_checkpointing="unsloth",
)
print("✓ Model prepared for training with Unsloth PEFT")

# =============================================================================
# PREPARE DATASET
# =============================================================================
# Data is already pre-formatted in the exact inference format
# Each sample has a "text" field with complete prompt→response

train_dataset = Dataset.from_list(sft_dataset)
print(f"✓ Training dataset: {len(train_dataset)} samples")

# Verify data format
if len(train_dataset) > 0:
    sample_text = train_dataset[0]["text"]
    print(f"  Sample length: {len(sample_text)} chars")
    print(f"  Has <start_of_turn>: {'<start_of_turn>' in sample_text}")
    print(f"  Has ACTION: {'ACTION:' in sample_text}")

# =============================================================================
# SFT TRAINING CONFIGURATION
# =============================================================================
# Increased epochs and adjusted learning rate for better convergence

sft_config = SFTConfig(
    output_dir=SFT_OUTPUT_DIR,
    per_device_train_batch_size=4,  # Increased batch size
    gradient_accumulation_steps=2,  # Effective batch size = 8
    num_train_epochs=3,  # Increased from 1 for better learning
    learning_rate=5e-5,  # Lower LR for more stable training
    lr_scheduler_type="cosine",  # Cosine decay
    warmup_ratio=0.1,  # 10% warmup
    logging_steps=10,
    save_steps=50,
    max_seq_length=4096,  # Sufficient for SFT samples with thinking
    dataset_text_field="text",
    fp16=not torch.cuda.is_bf16_supported(), 
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",  # Memory-efficient optimizer
    seed=RANDOM_SEED,
)

print(f"✓ SFT Config:")
print(f"  Epochs: {sft_config.num_train_epochs}")
print(f"  Batch size: {sft_config.per_device_train_batch_size} x {sft_config.gradient_accumulation_steps} = {sft_config.per_device_train_batch_size * sft_config.gradient_accumulation_steps}")
print(f"  Learning rate: {sft_config.learning_rate}")
print(f"  Max seq length: {sft_config.max_seq_length}")

# =============================================================================
# RUN TRAINING
# =============================================================================

sft_trainer = SFTTrainer(
    model=model, 
    args=sft_config, 
    train_dataset=train_dataset, 
    processing_class=tokenizer
)

print("\n🎯 Starting SFT training...")
print("=" * 60)
train_result = sft_trainer.train()
print("=" * 60)
print("✓ SFT training complete")

# Save adapter
sft_trainer.save_model(SFT_ADAPTER_DIR)
print(f"✓ SFT adapter saved to: {SFT_ADAPTER_DIR}")

# =============================================================================
# TRAINING LOSS ANALYSIS
# =============================================================================
# This helps verify the model is actually learning

print(f"\n{'='*70}")
print("📉 TRAINING LOSS ANALYSIS")
print(f"{'='*70}")

# Get training metrics from trainer
if hasattr(sft_trainer, 'state') and hasattr(sft_trainer.state, 'log_history'):
    log_history = sft_trainer.state.log_history
    
    # Extract loss values
    losses = [(entry.get('step', i), entry['loss']) 
              for i, entry in enumerate(log_history) if 'loss' in entry]
    
    if losses:
        steps, loss_values = zip(*losses)
        initial_loss = loss_values[0]
        final_loss = loss_values[-1]
        loss_reduction = (initial_loss - final_loss) / initial_loss * 100
        
        print(f"  📊 Initial Loss: {initial_loss:.4f}")
        print(f"  📊 Final Loss: {final_loss:.4f}")
        print(f"  📊 Loss Reduction: {loss_reduction:.1f}%")
        print(f"  📊 Training Steps: {len(losses)}")
        
        # Simple ASCII visualization
        if len(losses) >= 5:
            print(f"\n  Loss Trend (first 5 → last 5):")
            first5 = loss_values[:5]
            last5 = loss_values[-5:]
            print(f"    Start: {' → '.join(f'{l:.3f}' for l in first5)}")
            print(f"    End:   {' → '.join(f'{l:.3f}' for l in last5)}")
        
        # Warning if loss didn't decrease much
        if loss_reduction < 10:
            print(f"\n  ⚠️ WARNING: Loss only decreased {loss_reduction:.1f}%")
            print(f"     Consider: more epochs, different LR, or check data format")
    else:
        print("  ⚠️ No loss values recorded in training")
else:
    print("  ⚠️ Training state not available")

print(f"{'='*70}")


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


🧹 Cleaned up: ./sft_output
🧹 Cleaned up: ./sft_adapter
✓ Previous SFT artifacts cleaned


Unsloth 2025.12.9 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✓ Model prepared for training with Unsloth PEFT
✓ Training dataset: 639 samples
  Sample length: 1171 chars
  Has <start_of_turn>: True
  Has ACTION: True
✓ SFT Config:
  Epochs: 3
  Batch size: 4 x 2 = 8
  Learning rate: 5e-05
  Max seq length: 4096


Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/639 [00:00<?, ? examples/s]

...


🎯 Starting SFT training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 639 | Num Epochs = 3 | Total steps = 240
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.795900
20,1.875900
30,0.860900
40,0.372800
50,0.252900
60,0.250000
70,0.219800
80,0.199200
90,0.191000
100,0.184200


✓ SFT training complete
✓ SFT adapter saved to: ./sft_adapter

📉 TRAINING LOSS ANALYSIS
  📊 Initial Loss: 2.7959
  📊 Final Loss: 0.1507
  📊 Loss Reduction: 94.6%
  📊 Training Steps: 24

  Loss Trend (first 5 → last 5):
    Start: 2.796 → 1.876 → 0.861 → 0.373 → 0.253
    End:   0.141 → 0.135 → 0.144 → 0.145 → 0.151


In [15]:
# =============================================================================
# POST-SFT MODEL FORMAT TEST
# =============================================================================
# Quick test to verify the model outputs the correct format after training
# Run this before full evaluation to catch format issues early

print("🔬 POST-SFT MODEL FORMAT TEST")
print("=" * 70)

# Switch to inference mode
FastLanguageModel.for_inference(model)

# Create a simple test prompt matching inference format
test_account = "TEST-123456"
test_prompt = f"""<start_of_turn>user
Investigate account {test_account} for money laundering.

{TOOL_DESC}

CURRENT STATUS:
Progress: 0 accounts | $0 traced
Path: (starting)
Strategy: EXPLORE<end_of_turn>
<start_of_turn>model
"""

# Generate
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=False)
if "<end_of_turn>" in response:
    response = response.split("<end_of_turn>")[0]

print("📤 MODEL OUTPUT:")
print("-" * 40)
print(response)
print("-" * 40)

# Check format
has_action = "ACTION:" in response
has_params = "PARAMS:" in response
tool_calls = extract_tool_calls(response)

print(f"\n✓ Has ACTION: {has_action}")
print(f"✓ Has PARAMS: {has_params}")
print(f"✓ Extracted tool calls: {tool_calls}")

if tool_calls:
    print(f"\n✅ FORMAT OK - Model outputs valid tool calls")
else:
    print(f"\n⚠️ FORMAT ISSUE - Model output not parseable")
    print(f"   SFT may need more training or data format adjustment")

print("=" * 70)


🔬 POST-SFT MODEL FORMAT TEST
📤 MODEL OUTPUT:
----------------------------------------
<thinking>Starting investigation. First step is to get account summary to assess risk level.</thinking>
ACTION: get_account_summary
PARAMS: {"account_id": "TEST-123456"}
----------------------------------------

✓ Has ACTION: True
✓ Has PARAMS: True
✓ Extracted tool calls: [{'name': 'get_account_summary', 'arguments': {'account_id': 'TEST-123456'}}]

✅ FORMAT OK - Model outputs valid tool calls


In [16]:
# Post-SFT evaluation
FastLanguageModel.for_inference(model)
post_sft_results = run_evaluation(model, tokenizer, "Post-SFT", n_episodes=EVAL_EPISODES)
all_eval_results.append(post_sft_results)

# =============================================================================
# COMPREHENSIVE COMPARISON: BASELINE vs POST-SFT
# =============================================================================
print(f"\n{'='*70}")
print("📊 BASELINE vs POST-SFT COMPARISON")
print(f"{'='*70}")

metrics = [
    ("Success Rate", "success_rate", "%", 100),
    ("Fallback Rate", "fallback_rate", "%", 100),
    ("Avg Steps", "avg_steps", "", 1),
    ("Strategy Quality", "strategy_quality", "/5", 1),
    ("Context Usage", "context_usage", "/5", 1),
    ("Trail Following", "trail_following", "/5", 1),
    ("Efficiency", "efficiency", "/5", 1),
]

print(f"{'Metric':<20} {'Baseline':<12} {'Post-SFT':<12} {'Change':<12}")
print("-" * 56)

for name, key, suffix, multiplier in metrics:
    baseline_val = baseline_results.get(key, 0) * multiplier
    post_sft_val = post_sft_results.get(key, 0) * multiplier
    change = post_sft_val - baseline_val
    change_str = f"{change:+.1f}{suffix}"
    
    # Color-code: success rate up is good, fallback rate down is good
    if key == "fallback_rate":
        indicator = "📉" if change < 0 else "📈" if change > 0 else "➡️"
    else:
        indicator = "📈" if change > 0 else "📉" if change < 0 else "➡️"
    
    print(f"{name:<20} {baseline_val:.1f}{suffix:<6} {post_sft_val:.1f}{suffix:<6} {indicator} {change_str}")

print(f"{'='*70}")

# Key insight
if post_sft_results.get('fallback_rate', 1) < baseline_results.get('fallback_rate', 1):
    print("✅ Fallback rate decreased - model is producing more valid tool calls!")
else:
    print("⚠️ Fallback rate did not decrease - model may need more training")

if post_sft_results['success_rate'] > baseline_results['success_rate']:
    print("✅ Success rate improved!")
elif post_sft_results['success_rate'] == baseline_results['success_rate']:
    print("➡️ Success rate unchanged - may need more/different training data")
else:
    print("⚠️ Success rate decreased - training may have caused regression")



🧪 EVALUATION: Post-SFT (MLflow Experiment: AML_Investigation_Post-SFT)

────────────────────────────────────────────────────────────
  Episode [1/10]: 136009-80D626250
────────────────────────────────────────────────────────────

🔍 LANGGRAPH INVESTIGATION: 136009-80D626250
   Memory managed by: LangGraph MemorySaver


2025/12/28 13:48:10 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: Yaml file './mlruns/552162325398870797/traces/tr-8b9d2434e465e150bd9c66b3ad3c2d6d/trace_info.yaml' exists as './mlruns/552162325398870797/traces/tr-8b9d2434e465e150bd9c66b3ad3c2d6d/trace_info.yaml



┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Starting investigation. First step is to get account summary to assess risk level.</thinki...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 136009-80D626250
Risks: HIGH_RISK_SCORE
S
│ 🔧 TOOL: get_account_summary(account_id=136009-80D626250)
│ 📤 RESULT: {"account_id": "136009-80D626250", "account_type": "Corporate", "entity_name": "...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Account is corporate, but risk score is high (0.81). Need to trace money flow by checking ...
│ 🧠 MEMORY (LangGraph state): Progress: 3 accounts | $12,357 traced
Path: 136009-80D626250 → 235676-80D62AD70 
│ 🔧 TOOL: get_recent_transactions(account_id=136009-80D626250, direction=outgoing)
│ 📤 RESULT: [{"to_account": "235676-80D62AD70", "amount": 9765.64, "currency": "Canadian Dol...

┌─ Step 3 ───────────────────────────────────────────────────────
│ 📝 MODEL: <t

2025/12/28 13:48:32 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: Yaml file './mlruns/552162325398870797/traces/tr-ec1b8ca1f91e1d4c1ff49b7889463e85/trace_info.yaml' exists as './mlruns/552162325398870797/traces/tr-ec1b8ca1f91e1d4c1ff49b7889463e85/trace_info.yaml



┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Starting investigation. First step is to get account summary to assess risk level.</thinki...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 11-800B654E0
Strategy: EXPLORE
│ 🔧 TOOL: get_account_summary(account_id=11-800B654E0)
│ 📤 RESULT: {"account_id": "11-800B654E0", "account_type": "Sole Proprietorship", "entity_na...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Account is not sanctioned, but has high risk score. Need to trace money flow by checking o...
│ 🧠 MEMORY (LangGraph state): Progress: 5 accounts | $16,969 traced
Path: 11314-814D8DE50 → 0-800B65530 → 4308
│ 🔧 TOOL: get_recent_transactions(account_id=11-800B654E0, direction=outgoing)
│ 📤 RESULT: [{"to_account": "11-800B654E0", "amount": 6866.15, "currency": "US Dollar"}, {"t...

┌─ Step 3 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Multiple la

2025/12/28 13:48:48 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: Yaml file './mlruns/552162325398870797/traces/tr-3da9c2a90ed42f1a3d4cbf374eb93eff/trace_info.yaml' exists as './mlruns/552162325398870797/traces/tr-3da9c2a90ed42f1a3d4cbf374eb93eff/trace_info.yaml



┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Starting investigation. First step is to get account summary to assess risk level.</thinki...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 1776-806EF8480
Risks: HIGH_RISK_SCORE
Str
│ 🔧 TOOL: get_account_summary(account_id=1776-806EF8480)
│ 📤 RESULT: {"account_id": "1776-806EF8480", "account_type": "Partnership", "entity_name": "...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Account is not sanctioned, but has high risk score. Need to trace money flow by checking o...
│ 🧠 MEMORY (LangGraph state): Progress: 2 accounts | $20,410,204 traced
Path: 1776-806EF8480 → 218587-806EF857
│ 🔧 TOOL: get_recent_transactions(account_id=1776-806EF8480, direction=outgoing)
│ 📤 RESULT: [{"to_account": "218587-806EF8570", "amount": 20410203.68, "currency": "Euro"}]...

┌─ Step 3 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinki

## 12. Reinforcement Learning with GRPO

Optimize the agent's investigation strategy using Group Relative Policy Optimization.


In [19]:
from trl import GRPOTrainer, GRPOConfig

def compute_reward(episode: InvestigationEpisode) -> float:
    """Reward function: Success + Efficiency - Penalties."""
    reward = 0.0
    if episode.terminated and episode.final_result:
        if episode.success:
            reward += 1.0
            reward += 0.1 * (MAX_STEPS - len(episode.steps))  # Efficiency bonus
        else:
            reward -= 0.5  # False positive
    else:
        reward -= 0.2
    
    # Loop penalty
    seen = set()
    for step in episode.steps:
        sig = f"{step['tool_call'].get('name')}:{step['tool_call'].get('arguments')}"
        if sig in seen: reward -= 0.1
        seen.add(sig)
    
    return reward

def generate_rl_prompts(n: int = 20) -> List[str]:
    accounts = get_investigation_start_accounts(n)
    return [f"<start_of_turn>user\nInvestigate {acc} for money laundering.<end_of_turn>\n<start_of_turn>model\n" for acc in accounts]

def reward_function(prompts: List[str], completions: List[str], **kwargs) -> List[float]:
    """
    Reward function for GRPO training.
    
    Args:
        prompts: List of input prompts
        completions: List of model completions
        **kwargs: Additional args from TRL (completion_ids, etc.)
    
    Returns:
        List of reward scores
    """
    rewards = []
    for prompt, completion in zip(prompts, completions):
        match = re.search(r"Investigate (\S+) for", prompt)
        if not match:
            rewards.append(-1.0)
            continue
        
        account = match.group(1)
        tool_calls = extract_tool_calls(completion)
        
        episode = InvestigationEpisode(start_account=account)
        for call in tool_calls:
            result = execute_tool_call(call)
            episode.add_step(completion, call, result)
            if call.get('name') == 'submit_suspicious_activity_report':
                episode.terminated = True
                episode.final_result = result
                episode.success = result.get('correct_identification', False)
                break
        
        rewards.append(compute_reward(episode))
    return rewards

# RL dataset
rl_prompts = generate_rl_prompts(20)
rl_dataset = Dataset.from_dict({"prompt": rl_prompts})

print(f"✓ RL dataset: {len(rl_prompts)} prompts")


✓ RL dataset: 20 prompts


In [20]:
# GRPO Training
grpo_config = GRPOConfig(
    output_dir="./grpo_output", per_device_train_batch_size=2, num_train_epochs=1,
    learning_rate=1e-5, logging_steps=5, max_completion_length=512, num_generations=4,
    seed=RANDOM_SEED, fp16=not torch.cuda.is_bf16_supported(), bf16=torch.cuda.is_bf16_supported(),
)

grpo_trainer = GRPOTrainer(
    model=model, args=grpo_config, train_dataset=rl_dataset,
    processing_class=tokenizer, reward_funcs=reward_function,
)

print("🎮 Starting GRPO training...")
grpo_trainer.train()
print("✓ GRPO training complete")

grpo_trainer.save_model("./grpo_adapter")


The model is already on multiple devices. Skipping the move to device specified in `args`.


🎮 Starting GRPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)
/usr/local/lib/python3.12/dist-packages/unsloth/kernels/utils.py:970: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 2304], which does not match the required output shape [4, 1, 2304]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:31.)
  out = torch_matmul(X, W, out = out)


Step,Training Loss,reward,reward_std,train,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,tools / call_frequency,tools / failure_frequency,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,cispo_clip_ratio,rewards / reward_function / mean,rewards / reward_function / std
5,1478406448558899.250000,-0.200000,0.000000,0,227.150000,190.200000,262.000000,0.000000,227.150000,190.200000,262.000000,0,0,0,0,0,0,0,1478406463292021248.000000,0,-0.200000,0.000000
10,12655458.400000,-0.200000,0.000000,No Log,253.350000,182.600000,378.800000,0.100000,223.383334,182.600000,271.000000,No Log,No Log,No Log,No Log,No Log,No Log,No Log,12655459166.687500,No Log,-0.200000,0.000000
15,943734.900000,-0.200000,0.000000,No Log,305.400000,228.400000,386.400000,0.000000,305.400000,228.400000,386.400000,No Log,No Log,No Log,No Log,No Log,No Log,No Log,943734918.526758,No Log,-0.200000,0.000000
20,1703.798200,-0.200000,0.000000,No Log,287.050000,215.200000,363.200000,0.050000,277.533337,215.200000,343.200000,No Log,No Log,No Log,No Log,No Log,No Log,No Log,1703798.021106,No Log,-0.200000,0.000000


/usr/local/lib/python3.12/dist-packages/unsloth/kernels/utils.py:970: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 2304], which does not match the required output shape [4, 1, 2304]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:31.)
  out = torch_matmul(X, W, out = out)


✓ GRPO training complete


## 13. Post-RL Evaluation


In [21]:
# Post-RL evaluation
FastLanguageModel.for_inference(model)
post_rl_results = run_evaluation(model, tokenizer, "Post-RL", n_episodes=EVAL_EPISODES)
all_eval_results.append(post_rl_results)

# Improvement summary
sft_gain = post_sft_results['success_rate'] - baseline_results['success_rate']
rl_gain = post_rl_results['success_rate'] - post_sft_results['success_rate']
total_gain = post_rl_results['success_rate'] - baseline_results['success_rate']

print(f"\n📈 Improvement Summary:")
print(f"  SFT gain: {sft_gain*100:+.1f}%")
print(f"  RL gain:  {rl_gain*100:+.1f}%")
print(f"  Total:    {total_gain*100:+.1f}%")


2025/12/28 14:02:15 INFO mlflow.tracking.fluent: Experiment with name 'AML_Investigation_Post-RL' does not exist. Creating a new experiment.



🧪 EVALUATION: Post-RL (MLflow Experiment: AML_Investigation_Post-RL)

────────────────────────────────────────────────────────────
  Episode [1/10]: 136009-80D626250
────────────────────────────────────────────────────────────

🔍 LANGGRAPH INVESTIGATION: 136009-80D626250
   Memory managed by: LangGraph MemorySaver

┌─ Step 1 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Starting investigation. First step is to get account summary to assess risk level.</thinki...
│ 🧠 MEMORY (LangGraph state): Progress: 1 accounts | $0 traced
Path: 136009-80D626250
Risks: HIGH_RISK_SCORE
S
│ 🔧 TOOL: get_account_summary(account_id=136009-80D626250)
│ 📤 RESULT: {"account_id": "136009-80D626250", "account_type": "Corporate", "entity_name": "...

┌─ Step 2 ───────────────────────────────────────────────────────
│ 📝 MODEL: <thinking>Account is corporate. Checking outgoing transactions is crucial to find laundering. </thin...
│ 🧠 MEMORY (LangGraph state): Progress: 3 accounts | 

## 14. Final Results Summary


In [22]:
# Create results summary
results_data = []
for result in all_eval_results:
    results_data.append({
        "Stage": result["stage"],
        "Success Rate": f"{result['success_rate']*100:.1f}%",
        "Avg Steps": f"{result['avg_steps']:.1f}",
        "Strategy": f"{result.get('strategy_quality', 0):.2f}/5",
        "Context": f"{result.get('context_usage', 0):.2f}/5",
        "Trail": f"{result.get('trail_following', 0):.2f}/5",
        "Efficiency": f"{result.get('efficiency', 0):.2f}/5",
        "Time": f"{result['total_time']:.1f}s"
    })

results_df = pd.DataFrame(results_data)

print("\n" + "="*90)
print("🏆 FINAL RESULTS: AML INVESTIGATION AGENT")
print("="*90)
print(f"\nModel: {MODEL_NAME}")
print(f"Evaluation Episodes: {EVAL_EPISODES}")
print(f"Max Steps: {MAX_STEPS}")
print("\n" + "-"*90)
display(results_df)

if len(all_eval_results) >= 3:
    print("\n📊 Improvements:")
    print(f"  Success: Baseline {all_eval_results[0]['success_rate']*100:.0f}% → Post-RL {all_eval_results[2]['success_rate']*100:.0f}%")
    print(f"  Strategy: {all_eval_results[0].get('strategy_quality',0):.1f} → {all_eval_results[2].get('strategy_quality',0):.1f}")
    print(f"  Context:  {all_eval_results[0].get('context_usage',0):.1f} → {all_eval_results[2].get('context_usage',0):.1f}")

print("\n" + "="*90)
print("✓ Evaluation complete!")



🏆 FINAL RESULTS: AML INVESTIGATION AGENT

Model: google/gemma-2-2b-it
Evaluation Episodes: 10
Max Steps: 50

------------------------------------------------------------------------------------------


,Stage,Success Rate,Avg Steps,Strategy,Context,Trail,Efficiency,Time
0,Baseline,10.0%,31.7,1.10/5,1.00/5,1.00/5,1.00/5,631.4s
1,Post-SFT,10.0%,6.9,1.50/5,1.40/5,1.40/5,1.40/5,117.7s
2,Post-RL,10.0%,6.3,1.50/5,1.40/5,1.40/5,1.40/5,111.0s



📊 Improvements:
  Success: Baseline 10% → Post-RL 10%
  Strategy: 1.1 → 1.5
  Context:  1.0 → 1.4

✓ Evaluation complete!
